In [5]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 20.7 ms (started: 2021-08-30 21:01:46 +08:00)


In [7]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #2 Aug 30, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main bba58a0] 更新 #2 Aug 30, 2021
 2 files changed, 816 insertions(+), 246 deletions(-)
 create mode 100644 "\345\270\270\350\247\201\347\224\250\344\276\213.ipynb"


To git@github.com:ustchope/pytorch_lightning_study.git
   ffbd8b5..bba58a0  main -> main


time: 5.19 s (started: 2021-08-30 21:02:03 +08:00)


In [6]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 1.04 ms (started: 2021-08-30 21:02:00 +08:00)


# 子模块

研究项目倾向于测试针对同一数据集的不同方法。 这在带有继承的 Lightning 中很容易做到。

例如，假设我们现在想要训练一个自动编码器用作 MNIST 图像的特征提取器。 我们正在从 LitMNIST 模块扩展我们的自动编码器，该模块已经定义了所有数据加载。 Autoencoder 模型中唯一改变的是初始化、转发、训练、验证和测试步骤。

In [ ]:
class Encoder(torch.nn.Module):
    pass


class Decoder(torch.nn.Module):
    pass


class AutoEncoder(LitMNIST):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.metric = MSE()

    def forward(self, x):
        return self.encoder(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch

        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        return loss

    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")

    def _shared_eval(self, batch, batch_idx, prefix):
        x, _ = batch
        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        self.log(f"{prefix}_loss", loss)

我们可以使用同一个训练器来训练它

In [ ]:
autoencoder = AutoEncoder()
trainer = Trainer()
trainer.fit(autoencoder)

请记住，forward 方法应该定义 LightningModule 的实际用途。 在这种情况下，我们想使用 AutoEncoder 来提取图像表示

In [ ]:
some_images = torch.Tensor(32, 1, 28, 28)
representations = autoencoder(some_images)

# 调试

以下是使调试更容易的标志。

## fast_dev_run

该标志通过运行 n 如果设置为 n (int) 否则为 1 如果设置为 True 训练和验证批次来运行“单元测试”。 关键是检测训练/验证循环中的任何错误，而不必等待完整的 epoch 崩溃。

（参见：Trainer 的 fast_dev_run 参数）

In [ ]:
# 运行 1 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=True)

# 运行 7 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=7)

> 此参数将禁用调谐器、检查点回调、提前停止回调、记录器和记录器回调（如 LearningRateLogger）并且仅运行 1 个时期。

## 检查梯度范数

日志（到记录器），每个权重矩阵的范数。

（参见：Trainer 的 track_grad_norm 参数）

In [ ]:
# the 2-norm
trainer = Trainer(track_grad_norm=2)

## 记录 GPU 使用情况

记录（到记录器）主机上每个 GPU 的 GPU 使用情况。

（参见：Trainer 的 log_gpu_memory 参数）

In [ ]:
trainer = Trainer(log_gpu_memory=True)

## 使模型过拟合数据子集

一个好的调试技术是取一小部分数据（比如每个类 2 个样本），并尝试让您的模型过度拟合。 如果不能，则表明它不适用于大型数据集。

（参见：Trainer 的 overfit_batches 参数）

In [ ]:
# 仅使用 1% 的训练数据（并在 val 和 test 中使用相同的训练数据加载器（关闭 shuffle））
trainer = Trainer(overfit_batches=0.01)

# 类似，但无论数据集大小如何，都有固定的 10 个批次
trainer = Trainer(overfit_batches=10)

有了这个标志，训练集、验证集和测试集都将是同一个训练集。 我们还将替换训练集中的采样器，为您关闭 shuffle。

## 打印 LightningModule 的摘要

每当 `.fit()` 函数被调用时，Trainer 将打印 LightningModule 的权重摘要。 默认情况下，它只打印顶级模块。 如果要显示网络中的所有子模块，请使用“完整”选项：

In [ ]:
trainer = Trainer(weights_summary="full")

您还可以通过在 LightningModule 中设置 `example_input_array` 属性来显示所有图层的中间输入和输出大小。 它将打印这样的表格

In [ ]:
  | Name  | Type        | Params | In sizes  | Out sizes
--------------------------------------------------------------
0 | net   | Sequential  | 132 K  | [10, 256] | [10, 512]
1 | net.0 | Linear      | 131 K  | [10, 256] | [10, 512]
2 | net.1 | BatchNorm1d | 1.0 K    | [10, 512] | [10, 512]

当您在 Trainer 上调用 .fit() 时。 这可以帮助您找到图层组合中的错误。
也可以看看：
* weights_summary参数
* ModelSummary

## 缩短Epoch

有时，仅使用一定百分比的训练、验证或测试数据（或一组批次）会很有帮助。 例如，您可以使用 20% 的训练集和 1% 的验证集。

在 Imagenet 等更大的数据集上，这可以帮助您比等待一个完整的 epoch 更快地调试或测试一些事情。

In [ ]:
# 仅使用 10% 的训练数据和 1% 的 val 数据
trainer = Trainer(limit_train_batches=0.1, limit_val_batches=0.01)

# 使用 10 批 train 和 5 批 val
trainer = Trainer(limit_train_batches=10, limit_val_batches=5)

## 设置验证健全性步骤的数量

Lightning 在训练开始时运行了几个验证步骤。 这可以避免在验证循环中崩溃，进入冗长的训练循环。

（参见：训练师的 num_sanity_val_steps 参数）

In [ ]:
# DEFAULT
trainer = Trainer(num_sanity_val_steps=2)

# Early stopping

## 提前停止一个Epoch

您可以通过覆盖 `on_train_batch_start()` 在满足某些条件时返回 -1 来提前停止一个纪元。

如果您重复执行此操作，对于您最初请求的每个 epoch，那么这将停止您的整个运行。

## 使用 EarlyStopping 回调基于指标提前停止

EarlyStopping 回调可用于监控验证指标并在未观察到改进时停止训练。

要启用它：
* 导入 EarlyStopping 回调。
* 使用 log() 方法记录要监控的指标。
* 初始化回调，并将监视器设置为您选择的记录指标。
* 将 EarlyStopping 回调传递给 Trainer 回调标志。

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


def validation_step(self):
    self.log("val_loss", loss)


trainer = Trainer(callbacks=[EarlyStopping(monitor="val_loss")])

您可以通过更改其参数来自定义回调行为。

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")
trainer = Trainer(callbacks=[early_stop_callback])

在极端点停止训练的其他参数：
* stop_threshold：一旦监控的数量达到这个阈值，立即停止训练。 当我们知道超出某个最佳值不会进一步使我们受益时，这很有用。
* divergence_threshold：一旦监控的数量变得比这个阈值更差，就停止训练。 当达到如此糟糕的值时，我们认为模型无法再恢复，最好提前停止并在不同的初始条件下运行。
* check_finite：开启后，如果监控指标变为 NaN 或无穷大，我们将停止训练。

如果您需要在训练的不同部分提前停止，请子类 EarlyStopping 并更改它的调用位置：

In [ ]:
class MyEarlyStopping(EarlyStopping):
    def on_validation_end(self, trainer, pl_module):
        # 覆盖它以在 val 循环结束时禁用提前停止
        pass

    def on_train_end(self, trainer, pl_module):
        # i相反，在训练循环结束时进行
        self._run_early_stopping_check(trainer, pl_module)

> EarlyStopping 回调在每个验证时期结束时运行，在默认配置下，在每个训练时期之后发生。 但是，可以通过在 Trainer 中设置各种参数来修改验证频率，例如 check_val_every_n_epoch 和 val_check_interval。 必须注意，耐心参数计算没有改进的验证时期的数量，而不是训练时期的数量。 因此，在参数 check_val_every_n_epoch=10 和pattern=3 的情况下，训练器在停止之前将执行至少 40 个训练 epoch。

# 超参数

Lightning 具有与命令行 ArgumentParser 无缝交互的实用程序，并与您选择的超参数优化框架配合得很好。

## ArgumentParser

In [ ]:
from argparse import ArgumentParser

parser = ArgumentParser()
parser.add_argument("--layer_1_dim", type=int, default=128)
args = parser.parse_args()

这允许您像这样调用您的程序：

In [ ]:
python trainer.py --layer_1_dim 64

## Argparser 最佳实践

最佳做法是将您的arguments分为三个部分。
* 训练器参数（gpus、num_nodes 等...）
* 模型特定参数（layer_dim、num_layers、learning_rate 等...）
* 程序参数（data_path、cluster_email 等...）

我们可以这样做。 首先，在您的 LightningModule 中，定义特定于该模块的参数。 请记住，数据拆分或数据路径也可能特定于某个模块（即：如果您的项目有一个在 Imagenet 上训练的模型和另一个在 CIFAR-10 上训练的模型）。

In [ ]:
class LitModel(LightningModule):
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("LitModel")
        parser.add_argument("--encoder_layers", type=int, default=12)
        parser.add_argument("--data_path", type=str, default="/some/path")
        return parent_parser

现在，在您的主训练文件中，添加 Trainer args、程序 args，并添加模型 args

In [ ]:
# ----------------
# trainer_main.py
# ----------------
from argparse import ArgumentParser

parser = ArgumentParser()

# 添加程序级别参数
parser.add_argument("--conda_env", type=str, default="some_name")
parser.add_argument("--notification_email", type=str, default="will@email.com")

# 添加特定于模型的参数
parser = LitModel.add_model_specific_args(parser)

# 将所有可用的训练器选项添加到 argparse
# ie: now --gpus --num_nodes ... --fast_dev_run all work in the cli
parser = Trainer.add_argparse_args(parser)

args = parser.parse_args()

现在你可以像这样调用运行你的程序：

In [ ]:
python trainer_main.py --gpus 2 --num_nodes 2 --conda_env 'my_env' --encoder_layers 12

最后，确保像这样开始训练：

In [ ]:
# 像这样初始化训练器
trainer = Trainer.from_argparse_args(args, early_stopping_callback=...)

# 不要这样
trainer = Trainer(gpus=hparams.gpus, ...)

# 直接用 Namespace 初始化模型
model = LitModel(args)

# 或使用所有键值对初始化模型
dict_args = vars(args)
model = LitModel(**dict_args)

## LightningModule 超参数

很多时候，我们训练一个模型的多个版本。 您可能会分享该模型或在几个月后再次使用该模型，此时了解该模型是如何训练的（即：什么学习率、神经网络等）非常有用。

Lightning 有几种方法可以将这些信息保存在检查点和 yaml 文件中。 这里的目标是提高可读性和再现性。

1. 在 LightningModule `__init__` 函数中使用 `save_hyperparameters()` 将使 Lightning 能够将所有提供的参数存储在 `self.hparams` 属性中。 这些超参数也将存储在模型检查点中，这简化了生产设置中的模型重新实例化。 这也使这些值可以通过 `self.hparams` 获得。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, layer_1_dim=128, learning_rate=1e-2, **kwargs):
        super().__init__()
        # 调用它以将 (layer_1_dim=128, learning_rate=1e-4) 保存到检查点
        self.save_hyperparameters()

        # equivalent
        self.save_hyperparameters("layer_1_dim", "learning_rate")

        # 现在可以从 hparams 访问 layer_1_dim
        self.hparams.layer_1_dim

2. 有时，您的 init 可能具有您可能不想保存的对象或其他参数。 在这种情况下，只选择几个

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, loss_fx, generator_network, layer_1_dim=128, **kwargs):
        super().__init__()
        self.layer_1_dim = layer_1_dim
        self.loss_fx = loss_fx

        # 调用它以将 (layer_1_dim=128) 保存到检查点
        self.save_hyperparameters("layer_1_dim")


# 加载指定其他参数
model = LitMNIST.load_from_checkpoint(PATH, loss_fx=torch.nn.SomeOtherLoss, 
                                      generator_network=MyGenerator())

3. 您还可以将 dict 或 Namespace 等完整对象转换为 hparams，以便将它们保存到检查点。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, conf: Optional[Union[Dict, Namespace, DictConfig]] = None, **kwargs):
        super().__init__()
        # 保存配置和任何额外的参数
        self.save_hyperparameters(conf)
        self.save_hyperparameters(kwargs)

        self.layer_1 = nn.Linear(28 * 28, self.hparams.layer_1_dim)
        self.layer_2 = nn.Linear(self.hparams.layer_1_dim, self.hparams.layer_2_dim)
        self.layer_3 = nn.Linear(self.hparams.layer_2_dim, 10)


conf = {...}
# OR
# conf = parser.parse_args()
# OR
# conf = OmegaConf.create(...)
model = LitMNIST(conf=conf, anything=10)

# 现在可以从 hparams 访问任何存储的变量
model.hparams.anything

# 为此，您需要使用 `self.hparams.layer_1_dim` 访问，而不是 `conf.layer_1_dim`
model = LitMNIST.load_from_checkpoint(PATH)

# Trainer args

回顾一下，将所有可能的训练器标志添加到 argparser 并以这种方式初始化训练器

In [ ]:
parser = ArgumentParser()
parser = Trainer.add_argparse_args(parser)
hparams = parser.parse_args()

trainer = Trainer.from_argparse_args(hparams)

# 或者如果您需要传入回调
trainer = Trainer.from_argparse_args(hparams, checkpoint_callback=..., callbacks=[...])

# 多个PL模块

我们经常有多个LightningModule，每个模块都有不同的参数。 LightningModule 允许您为每个文件定义参数，而不是污染 main.py 文件。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, layer_1_dim, **kwargs):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, layer_1_dim)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("LitMNIST")
        parser.add_argument("--layer_1_dim", type=int, default=128)
        return parent_parser

In [ ]:
class GoodGAN(LightningModule):
    def __init__(self, encoder_layers, **kwargs):
        super().__init__()
        self.encoder = Encoder(layers=encoder_layers)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("GoodGAN")
        parser.add_argument("--encoder_layers", type=int, default=12)
        return parent_parser

现在我们可以允许每个模型在 main.py 中注入它需要的参数

In [ ]:
def main(args):
    dict_args = vars(args)

    # 选择模型
    if args.model_name == "gan":
        model = GoodGAN(**dict_args)
    elif args.model_name == "mnist":
        model = LitMNIST(**dict_args)

    trainer = Trainer.from_argparse_args(args)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser = Trainer.add_argparse_args(parser)

    # 确定使用哪个模型
    parser.add_argument("--model_name", type=str, default="gan", help="gan or mnist")

    # 这条线是拉型号名的关键
    temp_args, _ = parser.parse_known_args()

    # 让模型添加它想要的东西
    if temp_args.model_name == "gan":
        parser = GoodGAN.add_model_specific_args(parser)
    elif temp_args.model_name == "mnist":
        parser = LitMNIST.add_model_specific_args(parser)

    args = parser.parse_args()

    # train
    main(args)

现在我们可以使用命令行界面训练 MNIST 或 GAN！

In [ ]:
$ python main.py --model_name gan --encoder_layers 24
$ python main.py --model_name mnist --layer_1_dim 128

# 生产中的推理

PyTorch Lightning 简化了将模型部署到生产中的过程。

## 导出到 ONNX

PyTorch Lightning 提供了一个方便的功能，可以将您的模型快速导出为 ONNX 格式，这使得模型可以独立于 PyTorch 并在 ONNX Runtime 上运行。

要将模型导出为 ONNX 格式，请使用文件路径和 input_sample 调用 Lightning 模块上的 to_onnx 函数。

In [ ]:
filepath = "model.onnx"
model = SimpleModel()
input_sample = torch.randn((1, 64))
model.to_onnx(filepath, input_sample, export_params=True)

如果您的 LightningModule 中指定了 `example_input_array` 属性，您也可以跳过输入样本的传递。

导出模型后，您可以通过以下方式在 ONNX 运行时运行它：

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)
input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: np.random.randn(1, 64).astype(np.float32)}
ort_outs = ort_session.run(None, ort_inputs)

## 导出到 TorchScript

TorchScript 允许您以可以在非 Python 环境中加载的方式序列化模型。 LightningModule 有一个方便的方法 to_torchscript() 返回一个脚本模块，您可以保存或直接使用它。

In [ ]:
model = SimpleModel()
script = model.to_torchscript()

# 保存在生产环境中使用
torch.jit.save(script, "model.pt")

建议您安装最新支持的 PyTorch 版本以不受限制地使用此功能。

# Lightning CLI 和配置文件

Lightning 可以帮助减少的另一个样板代码来源是命令行工具的实现。 此外，它还提供了一种使用单个文件配置实验的标准化方法，该文件包括 Trainer 的设置以及用户扩展的 LightningModule 和 LightningDataModule 类。 完整的配置会自动保存在日志目录中。 这样做的好处是大大简化了实验的重现性。

使用户扩展类可配置的主要要求是所有相关的 init 参数必须具有类型提示。 这不是一个非常苛刻的要求，因为无论如何都是很好的做法。 如果参数在文档字符串中描述，那么命令行工具的帮助将显示它们。

## LightningCLI

训练命令行工具的实现是通过 LightningCLI 类完成的。 pytorch-lightning 的最小安装不包括此支持。 要启用它，请将 Lightning 安装为 pytorch-lightning[extra] 或安装包 jsonargparse[signatures]。

在用户的 LightningModule 类实现所有必需的 `*_dataloader` 方法的情况下，trainer.py 工具可以像这样简单：

In [ ]:
cli = LightningCLI(MyModel)

可以通过运行 `python trainer.py --help` 来显示描述所有可配置选项和默认值的工具的帮助。 可以通过提供单独的命令行参数来更改默认选项。 但是，更好的做法是创建一个配置文件并将其提供给工具。 一种方法是：

In [ ]:
# 转储默认配置以作为参考
python trainer.py fit --print_config > config.yaml
# 根据您的喜好修改配置 - 您可以删除所有默认参数
nano config.yaml
# 使用配置适合您的模型
python trainer.py fit --config config.yaml

LightningCLI 类的实例化负责解析命令行和配置文件选项、实例化类、设置回调以将配置保存在日志目录中并最终运行训练器。 例如，生成的对象 cli 可用于获取模型的实例 (cli.model)。

在使用不同配置进行多次实验后，每个实验都会在其各自的日志目录中拥有一个 config.yaml 文件。 该文件可用作参考以详细了解用于每个特定实验的所有设置，也可用于简单地重现训练，例如：

In [ ]:
python trainer.py fit --config lightning_logs/version_7/config.yaml

如果需要单独的 LightningDataModule 类，则训练器工具只需要进行如下小修改：

In [ ]:
cli = LightningCLI(MyModel, MyDataModule)

包括文档字符串中推荐的参数描述在内的 MyModel 的可能实现的开始可能是下面的一个。 请注意，通过使用类型提示和文档字符串，无需复制此信息来定义其可配置参数。

In [ ]:
class MyModel(LightningModule):
    def __init__(self, encoder_layers: int = 12, decoder_layers: List[int] = [2, 4]):
        """Example encoder-decoder model

        Args:
            encoder_layers: Number of layers for the encoder
            decoder_layers: Number of layers for each decoder block
        """
        super().__init__()
        self.save_hyperparameters()

使用此模型类，训练器工具的帮助如下所示：

In [ ]:
$ python trainer.py fit --help
usage: trainer.py [-h] [--config CONFIG] [--print_config [={comments,skip_null}+]] ...

optional arguments:
  -h, --help            Show this help message and exit.
  --config CONFIG       Path to a configuration file in json or yaml format.
  --print_config [={comments,skip_null}+]
                        Print configuration and exit.
  --seed_everything SEED_EVERYTHING
                        Set to an int to run seed_everything with this value before classes instantiation
                        (type: Optional[int], default: null)

Customize every aspect of training via flags:
  ...
  --trainer.max_epochs MAX_EPOCHS
                        Stop training once this number of epochs is reached. (type: Optional[int], default: null)
  --trainer.min_epochs MIN_EPOCHS
                        Force training for at least these many epochs (type: Optional[int], default: null)
  ...

Example encoder-decoder model:
  --model.encoder_layers ENCODER_LAYERS
                        Number of layers for the encoder (type: int, default: 12)
  --model.decoder_layers DECODER_LAYERS
                        Number of layers for each decoder block (type: List[int], default: [2, 4])

选项 --print_config 给出的默认配置是 yaml 格式，对于上面的示例，如下所示：

In [ ]:
$ python trainer.py fit --print_config
model:
  decoder_layers:
  - 2
  - 4
  encoder_layers: 12
trainer:
  accelerator: null
  accumulate_grad_batches: 1
  amp_backend: native
  amp_level: O2
  ...

请注意，每个类（模型和训练器）都有一个部分，包括该类的所有初始化参数。 此分组也用于前面显示的帮助的格式设置。

## 更改子命令

通过更改提供的子命令，CLI 支持从命令行运行任何训练器功能：

In [ ]:
$ python trainer.py --help
usage: trainer.py [-h] [--config CONFIG] [--print_config [={comments,skip_null}+]] {fit,validate,test,predict,tune} ...

pytorch-lightning trainer command line tool

optional arguments:
  -h, --help            Show this help message and exit.
  --config CONFIG       Path to a configuration file in json or yaml format.
  --print_config [={comments,skip_null}+]
                        Print configuration and exit.

subcommands:
  For more details of each subcommand add it as argument followed by --help.

  {fit,validate,test,predict,tune}
    fit                 Runs the full optimization routine.
    validate            Perform one evaluation epoch over the validation set.
    test                Perform one evaluation epoch over the test set.
    predict             Run inference on your data.
    tune                Runs routines to tune hyperparameters before training.
$ python trainer.py test --trainer.limit_test_batches=10 [...]b

## 命令行参数的使用

对于每个实现的 CLI，鼓励用户通过阅读使用 --help 选项打印的文档并使用 --print_config 选项来指导配置文件的编写来学习如何运行它。仅通过阅读帮助可能不清楚的更多细节如下。

LightningCLI 基于 argparse，因此遵循与许多 POSIX 命令行工具相同的参数样式。长选项以两个破折号为前缀，其对应的值应提供一个空格或等号，如--option value 或--option=value。命令行选项从左到右解析，因此如果设置出现多次，最右边的值将覆盖之前的值。如果一个类有一个 init 参数是必需的（即没有默认值），它会作为 --option 给出，这使得它更明确且更具可读性，而不是依赖于位置参数。

调用 CLI 时，可以使用单独的参数提供所有选项。但是，鉴于 CLI 具有大量选项，建议使用配置文件和单个参数的组合。因此，一个常见的模式可能是一个单一的配置文件，只有几个覆盖配置中的默认值或值的单独参数，例如：

In [ ]:
$ python trainer.py fit --config experiment_defaults.yaml --trainer.max_epochs 100

另一种常见模式可能是具有多个配置文件：

In [ ]:
$ python trainer.py --config config1.yaml --config config2.yaml test --config config3.yaml [...]

如前所述，首先解析 config1.yaml，然后解析 config2.yaml。 因此，如果在两个文件中都定义了单独的设置，那么将使用 config2.yaml 中的设置。 config1.yaml 中不在 config2.yaml 中的设置将被保留。 config3.yaml 也是如此。

子命令之前的配置文件（在本例中为 test）可以包含其中多个的自定义配置，例如：

In [ ]:
$ cat config1.yaml
fit:
    trainer:
        limit_train_batches: 100
        max_epochs: 10
test:
    trainer:
        limit_test_batches: 10

而在子命令之后传递的配置文件将是：

In [ ]:
$ cat config3.yaml
trainer:
    limit_train_batches: 100
    max_epochs: 10
# the argument passed to `trainer.test(ckpt_path=...)`
ckpt_path: "a/path/to/a/checkpoint"

选项组也可以作为独立的配置文件提供：

In [ ]:
$ python trainer.py fit --trainer trainer.yaml --model model.yaml --data data.yaml [...]

在集群中运行实验时，可能需要使用需要从远程位置访问的配置。 LightningCLI 带有 fsspec 支持，允许从多种类型的远程文件系统读取和写入。 一个例子是，如果您已经安装了 gcsfs，那么一个配置可以存储在 S3 存储桶中并通过以下方式访问：

In [ ]:
$ python trainer.py --config s3://bucket/config.yaml [...]

在某些情况下，人们可能会在环境变量中传递整个配置，这也可以用来代替文件的路径，例如：

In [ ]:
$ python trainer.py fit --trainer "$TRAINER_CONFIG" --model "$MODEL_CONFIG" [...]

环境变量的替代方法是使用 env_parse=True 实例化 CLI。 在这种情况下，帮助显示所有选项的环境变量名称。 PL_CONFIG 中将给出全局配置，不需要指定任何命令行参数。

也可以设置默认配置文件的路径。 如果文件存在，它将自动加载而无需指定任何命令行参数。 给出的参数将覆盖默认配置文件中的值。 在当前工作目录中加载默认文件 my_cli_defaults.yaml 将实现为：

In [ ]:
cli = LightningCLI(MyModel, MyDataModule, parser_kwargs={"default_config_files": ["my_cli_defaults.yaml"]})

或者如果您想要每个子命令的默认值：

In [ ]:
cli = LightningCLI(MyModel, MyDataModule, parser_kwargs={"fit": {"default_config_files": ["my_fit_defaults.yaml"]}})

要在用户的主目录中加载文件，只需更改为 ~/.my_cli_defaults.yaml。 请注意，此设置是通过 parser_kwargs 给出的。 支持更多参数。 有关详细信息，请参阅 ArgumentParser API 文档。

## 仅实例化模式

CLI 旨在以最少的代码更改开始拟合。 在类实例化时，CLI 将自动调用与提供的子命令关联的训练器函数，因此您不必这样做。 为避免这种情况，您可以设置以下参数：

In [ ]:
cli = LightningCLI(MyModel, run=False)  # True by default
# you'll have to call fit yourself:
cli.trainer.fit(cli.model)

在这种模式下，解析器中添加了子命令。 这对于实现自定义逻辑非常有用，而无需对 CLI 进行子类化，但仍使用 CLI 的实例化和参数解析功能。

## 训练器回调和类类型参数

Trainer 类的一个非常重要的参数是回调。 与其他只需要数字或字符串的更简单的参数相比，回调需要回调子类的实例列表。 要在配置文件中指定这种参数，每个回调必须作为一个字典给出，其中包含一个 class_path 条目和一个类的导入路径，以及一个可选的 init_args 条目，其中包含实例化它所需的参数。 因此，定义几个回调的简单配置文件示例如下：

In [ ]:
trainer:
  callbacks:
    - class_path: pytorch_lightning.callbacks.EarlyStopping
      init_args:
        patience: 5
    - class_path: pytorch_lightning.callbacks.LearningRateMonitor
      init_args:
        ...

与回调类似，Trainer 和用户扩展的 LightningModule 和 LightningDataModule 类中的任何参数都可以使用 class_path 和 init_args 以相同的方式配置为类型提示。

## 多个模型和/或数据集

在前面的示例中，LightningCLI 仅适用于单个模型和数据模块类。 但是，在很多情况下，目标是能够轻松地为多个模型和数据集运行许多实验。 对于这些情况，可以配置该工具，以便模型和/或数据模块由导入路径和 init 参数指定。 例如，使用一个工具实现为：

In [ ]:
cli = LightningCLI(MyModelBaseClass, MyDataModuleBaseClass, 
                   subclass_mode_model=True, subclass_mode_data=True)

一个可能的配置文件如下：

In [ ]:
model:
  class_path: mycode.mymodels.MyModel
  init_args:
    decoder_layers:
    - 2
    - 4
    encoder_layers: 12
data:
  class_path: mycode.mydatamodules.MyDataModule
  init_args:
    ...
trainer:
  callbacks:
    - class_path: pytorch_lightning.callbacks.EarlyStopping
      init_args:
        patience: 5
    ...

只允许作为 MyModelBaseClass 子类的模型类，同样只允许 MyDataModuleBaseClass 的子类。 如果给出 LightningModule 和 LightningDataModule 作为基类，那么该工具将允许任何闪电模块和数据模块。

> 请注意，对于子类模式，--help 选项不会显示特定子类的信息。 要获得子类的帮助，可以使用选项 --model.help 和 --data.help，后跟所需的类路径。 同样 --print_config 不包括特定子类的设置。 要包含它们，应在 --print_config 选项之前给出类路径。 帮助和打印配置的示例是：
>
>```
$ python trainer.py fit --model.help mycode.mymodels.MyModel
$ python trainer.py fit --model mycode.mymodels.MyModel --print_config
```

## 具有多个子模块的模型

许多用例需要有多个模块，每个模块都有自己的可配置选项。 使用 LightningCLI 处理此问题的一种可能方法是实现单个模块，将每个子模块作为初始化参数。 由于 init 参数的类型是类，因此在配置中将使用 class_path 和 init_args 条目指定这些参数。 例如，一个模型可以实现为：

In [ ]:
class MyMainModel(LightningModule):
    def __init__(self, encoder: EncoderBaseClass, decoder: DecoderBaseClass):
        """Example encoder-decoder submodules model

        Args:
            encoder: Instance of a module for encoding
            decoder: Instance of a module for decoding
        """
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

如果 CLI 实现为 LightningCLI(MyMainModel)，则配置如下：

In [ ]:
model:
  encoder:
    class_path: mycode.myencoders.MyEncoder
    init_args:
      ...
  decoder:
    class_path: mycode.mydecoders.MyDecoder
    init_args:
      ...

也可以组合 subclass_mode_model=True 和 submodules，从而有两个级别的 class_path。

## 自定义 LightningCLI

LightningCLI 类的 init 参数可用于自定义一些东西，即：工具的描述，启用环境变量的解析和附加参数以实例化训练器和配置解析器。

然而对于许多用例来说，init 参数是不够的。出于这个原因，该类被设计为可以扩展以自定义命令行工具的不同部分。 LightningCLI 使用的参数解析器类是 LightningArgumentParser，它是 Python 的 argparse 的扩展，因此可以使用 add_argument() 方法添加参数。与 argparse 相比，它具有添加参数的其他方法，例如 add_class_arguments() 从类的 init 中添加所有参数，但要求参数具有类型提示。有关这方面的更多详细信息，请参阅相应的文档。

LightningCLI 类具有 add_arguments_to_parser() 方法，可以实现该方法以包含更多参数。解析后，配置存储在类实例的 config 属性中。 LightningCLI 类还有两个方法可用于在训练器运行之前和之后运行代码：before_<subcommand> 和 after_<subcommand>。一个现实的例子是在执行之前和之后发送电子邮件。 fit 子命令的代码类似于：

In [ ]:
class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_argument("--notification_email", default="will@email.com")

    def before_fit(self):
        send_email(address=self.config["notification_email"], message="trainer.fit starting")

    def after_fit(self):
        send_email(address=self.config["notification_email"], message="trainer.fit finished")


cli = MyLightningCLI(MyModel)

请注意，配置对象 self.config 是一个字典，其键是全局选项或选项组。 它与前面描述的 yaml 格式具有相同的结构。 这意味着例如用于实例化训练器类的参数可以在 self.config['fit']['trainer'] 中找到。

> 查看 LightningCLI 类 API 参考，了解可扩展以自定义 CLI 的其他方法。

## 配置callbacks

如前所述，可以通过 class_path 和 init_args 条目将任何回调包含在配置中来添加。 但是，在其他情况下，回调应该始终存在并且是可配置的。 这可以按如下方式实现：

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping


class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_lightning_class_args(EarlyStopping, "my_early_stopping")
        parser.set_defaults({"my_early_stopping.patience": 5})


cli = MyLightningCLI(MyModel)

要在配置中更改 EarlyStopping 的配置，它将是：

In [ ]:
model:
  ...
trainer:
  ...
my_early_stopping:
  patience: 5

> 上面的示例覆盖了 add_arguments_to_parser 中的默认值。 包括在内是为了表明如果需要可以更改默认值。 但是，请注意，覆盖源代码中的默认值并不旨在用于在实验后存储任务的最佳超参数。 为了便于重现，源代码应该是稳定的。 更好的做法是将任务的最佳超参数存储在独立于源代码的配置文件中。

## 类类型默认值

对类作为类型提示的支持允许使用相同的 CLI 尝试多种可能性。 这是一个很有用的特性，但它可能会使使用类的实例作为默认值变得很诱人。 例如：

In [ ]:
class MyMainModel(LightningModule):
    def __init__(
        self,
        backbone: torch.nn.Module = MyModel(encoder_layers=24),  # BAD PRACTICE!
    ):
        super().__init__()
        self.backbone = backbone

通常类是可变的，就像在这种情况下一样。 MyModel 的实例将在第一次导入定义 MyMainModel 的模块时创建。 这意味着主干的默认值将在 CLI 类运行 seed_everything 之前被初始化，使其不可重现。 此外，如果在同一个 Python 进程中多次使用 MyMainModel 并且没有覆盖主干参数，则同一个实例将在多个地方使用，这很可能不是开发人员的意图。 将实例作为默认值也使得无法生成完整的配置文件，因为对于任意类，不知道使用哪些参数来实例化它。

这些问题的一个好的解决方案是没有默认值或将默认值设置为特殊值（例如字符串），该值将在 init 中检查并相应地实例化。 如果类参数没有默认值并且 CLI 是子类，则可以按如下方式设置默认值：

In [ ]:
default_backbone = {
    "class_path": "import.path.of.MyModel",
    "init_args": {
        "encoder_layers": 24,
    },
}


class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.set_defaults({"model.backbone": default_backbone})

避免编写字典的更紧凑版本是：

In [ ]:
from jsonargparse import lazy_instance


class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.set_defaults({"model.backbone": lazy_instance(MyModel, encoder_layers=24)})

## 参数链接

另一种可能需要扩展 LightningCLI 的情况是模型和数据模块依赖于一个公共参数。 例如，在某些情况下，两个类都需要知道 batch_size。 在配置文件中两次提供相同的值是一种负担和错误。 为了避免这种情况，可以配置解析器，以便只给出一次值，然后相应地传播。 使用如下所示的工具，batch_size 只需要在配置的数据部分提供。

In [ ]:
class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.link_arguments("data.batch_size", "model.batch_size")


cli = MyLightningCLI(MyModel, MyDataModule)

在工具的帮助中观察到参数的链接，对于这个例子，它看起来像：

In [ ]:
$ python trainer.py fit --help
  ...
    --data.batch_size BATCH_SIZE
                          Number of samples in a batch (type: int, default: 8)

  Linked arguments:
    model.batch_size <-- data.batch_size
                          Number of samples in a batch (type: int)

有时参数值仅在类实例化后可用。 例如，您的模型需要一定数量的类来实例化其全连接层（用于分类任务），但该值在实例化数据模块之前不可用。 下面的代码说明了如何解决这个问题。

In [ ]:
class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.link_arguments("data.num_classes", "model.num_classes", apply_on="instantiate")


cli = MyLightningCLI(MyClassModel, MyDataModule)

实例化链接用于自动确定实例化的顺序，在这种情况下数据在前。

> 参数的链接可用于更复杂的情况。 例如，通过将多个设置作为输入的函数来导出值。 有关更多详细信息，请查看 link_arguments 的 API。

## 优化器和学习率调度器

优化器和学习率调度器也可以配置。 最常见的情况是模型只有一个优化器和可选的单个学习率调度器。 在这种情况下，模型的 LightningModule 可以不实现 configure_optimizers 方法，因为它通常总是相同的，只是添加了样板。 下面的代码片段展示了如何实现它：

In [ ]:
import torch


class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_optimizer_args(torch.optim.Adam)
        parser.add_lr_scheduler_args(torch.optim.lr_scheduler.ExponentialLR)


cli = MyLightningCLI(MyModel)

有了这个， configure_optimizers 方法会自动实现，并且在配置中，优化器和 lr_scheduler 组将接受给定类的所有选项，在这个例子中是 Adam 和 ExponentialLR。 因此，配置文件的结构如下：

In [ ]:
optimizer:
  lr: 0.01
lr_scheduler:
  gamma: 0.2
model:
  ...
trainer:
  ...

这些参数中的任何一个都可以直接通过命令行传递。 例如：

In [ ]:
$ python trainer.py fit --optimizer.lr=0.01 --lr_scheduler.gamma=0.2

还可以通过将它们作为元组在多个类中进行选择。 例如：

In [ ]:
class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_optimizer_args((torch.optim.SGD, torch.optim.Adam))

在这种情况下，在配置优化器组而不是直接初始化设置，它应该指定 class_path 和可选的 init_args。 元组中类的子类也将被接受。 配置文件的相应示例是：

In [ ]:
optimizer:
  class_path: torch.optim.Adam
  init_args:
    lr: 0.01

同样通过命令行：

In [ ]:
$ python trainer.py fit --optimizer.class_path=torch.optim.Adam --optimizer.init_args.lr=0.01

可以通过链接配置组来禁用 configure_optimizers 的自动实现。 一个例子是 ReduceLROnPlateau，它需要指定一个监视器。 这将是：

In [ ]:
from pytorch_lightning.utilities.cli import instantiate_class


class MyModel(LightningModule):
    def __init__(self, optimizer_init: dict, lr_scheduler_init: dict):
        super().__init__()
        self.optimizer_init = optimizer_init
        self.lr_scheduler_init = lr_scheduler_init

    def configure_optimizers(self):
        optimizer = instantiate_class(self.parameters(), self.optimizer_init)
        scheduler = instantiate_class(optimizer, self.lr_scheduler_init)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "metric_to_track"}


class MyLightningCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_optimizer_args(
            torch.optim.Adam,
            link_to="model.optimizer_init",
        )
        parser.add_lr_scheduler_args(
            torch.optim.lr_scheduler.ReduceLROnPlateau,
            link_to="model.lr_scheduler_init",
        )


cli = MyLightningCLI(MyModel)

> 对于将 pytorch_lightning.utilities.cli.LightningArgumentParser.add_optimizer_args() 与单个类或类元组一起使用的两种可能性，提供给 optimizer_init 的值将始终是包含 class_path 和 init_args 条目的字典。 函数instantiate_class() 负责导入class_path 中定义的类并使用一些位置参数实例化它，在本例中为self.parameters() 和init_args。 使用 link_to 时可以添加任意数量的优化器和学习率调度器。

## 与再现性相关的注释

再现性的话题很复杂，仅仅提供一个人们可以以意想不到的方式使用的类是不可能保证再现性的。尽管如此，LightningCLI 试图提供一个框架和建议，以使可重复性更简单。

运行实验时，最好使用源代码的稳定版本，可以是已发布的包，也可以至少是某个版本控制存储库的提交。对于 CLI 的每次运行，配置文件会自动保存，包括所有设置。这对于在不需要查看源代码的情况下找出特定运行执行的操作非常有用。如果错误地丢失了源代码的确切版本或更改了某些默认值，则拥有完整配置意味着保留了大部分信息。

该类的目标是实现 CLI，因为从 shell 运行命令提供了与 Python 源代码的分离。理想情况下，CLI 将作为稳定包安装的一部分放置在您的路径中，而不是从可能具有未提交的本地修改的存储库的克隆中运行。创建包含 CLI 的可安装包超出了本文档的范围。这只是作为为那些努力争取最佳实践的人的预告而提及的。

# 学习率查找器

对于训练深度神经网络，选择一个好的学习率对于更好的性能和更快的收敛都是必不可少的。 即使是像 Adam 这样自我调整学习率的优化器也可以从更多的最优选择中受益。

为了减少关于选择一个好的初始学习率的猜测量，可以使用学习率查找器。 如本文所述，学习率查找器进行了一次小规模运行，其中在每个处理批次后增加学习率，并记录相应的损失。 其结果是 lr 与损失图，可用作选择最佳初始 lr 的指导。

> 目前，此功能仅适用于具有单个优化器的模型。 LR Finder 对 DDP 及其任何变体的支持尚未实现。 它即将到来。

## 使用 Lightning 的内置  LR finder

要启用学习率查找器，您的闪电模块需要具有 learning_rate 或 lr 属性。 然后，在trainer构建过程中设置Trainer(auto_lr_find=True)，然后调用trainer.tune(model)来运行LR finder。 建议的 learning_rate 将写入控制台并自动设置到您的闪电模块，可以通过 self.learning_rate 或 self.lr 访问。

In [ ]:
class LitModel(LightningModule):
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=(self.lr or self.learning_rate))


model = LitModel()

# finds learning rate 自动将 hparams.lr 或 hparams.learning_rate 设置为该学习率
trainer = Trainer(auto_lr_find=True)

trainer.tune(model)

如果您的模型使用任意值而不是 self.lr 或 self.learning_rate，请将该值设置为 auto_lr_find：

In [ ]:
model = LitModel()

# to set to your own hparams.my_value
trainer = Trainer(auto_lr_find="my_value")

trainer.tune(model)

您还可以检查学习率查找器的结果或只是玩弄算法的参数。 这可以通过调用 lr_find() 方法来完成。 一个典型的例子如下：

In [ ]:
model = MyModelClass(hparams)
trainer = Trainer()

# Run learning rate finder
lr_finder = trainer.tuner.lr_find(model)

# Results can be found in
lr_finder.results

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

# update hparams of the model
model.hparams.lr = new_lr

# Fit model
trainer.fit(model)

lr_finder.plot() 生成的图应该类似于下图。 建议不要选择损失最低的学习率，而是选择最陡峭的下降斜率（红点）中间的某个值。 这是返回 py lr_finder.suggestion() 的点。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtzvas4nhbj613f0u0ab902.jpg)

该算法的参数可以在下面看到。

```
pytorch_lightning.tuner.lr_finder.lr_find(trainer, model, min_lr=1e-08, max_lr=1, num_training=100, mode='exponential', early_stop_threshold=4.0, update_attr=False)
```

# LOGGERS

Lightning 支持最流行的日志框架（TensorBoard、Comet 等）。 默认情况下使用 TensorBoard，但您可以将以下记录器的任意组合传递给训练器。

> 默认情况下，所有记录器都记录到 os.getcwd()。 在不创建记录器集的情况下更改路径 Trainer(default_root_dir='/your/path/to/save/checkpoints')

阅读有关日志记录选项的更多信息。

要记录图像或音频样本等任意工件，请使用 trainer.log_dir 属性来解析路径。

In [ ]:
def training_step(self, batch, batch_idx):
    img = ...
    log_image(img, self.trainer.log_dir)

## Comet.ml

Comet.ml 是第三方记录器。 要使用 CometLogger 作为您的记录器，请执行以下操作。 首先，安装软件包：

In [ ]:
pip install comet-ml

然后配置记录器并将其传递给训练器：

In [ ]:
import os
from pytorch_lightning.loggers import CometLogger

comet_logger = CometLogger(
    api_key=os.environ.get("COMET_API_KEY"),
    workspace=os.environ.get("COMET_WORKSPACE"),  # Optional
    save_dir=".",  # Optional
    project_name="default_project",  # Optional
    rest_api_key=os.environ.get("COMET_REST_API_KEY"),  # Optional
    experiment_name="default",  # Optional
)
trainer = Trainer(logger=comet_logger)

CometLogger 在你的 LightningModule 中除了 `__init__ `之外的任何地方都可用。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        self.logger.experiment.add_image("generated_images", some_img, 0)

## MLflow

MLflow 是第三方记录器。 要使用 MLFlowLogger 作为您的记录器，请执行以下操作。 首先，安装软件包：

In [ ]:
pip install mlflow

然后配置记录器并将其传递给训练器：

In [ ]:
from pytorch_lightning.loggers import MLFlowLogger

mlf_logger = MLFlowLogger(experiment_name="default", tracking_uri="file:./ml-runs")
trainer = Trainer(logger=mlf_logger)

## Neptune.ai

Neptune.ai 是第三方记录器。 要将 NeptuneLogger 用作记录器，请执行以下操作。 首先，安装软件包：

In [ ]:
pip install neptune-client

然后配置记录器并将其传递给训练器：

In [ ]:
from pytorch_lightning.loggers import NeptuneLogger

neptune_logger = NeptuneLogger(
    api_key="ANONYMOUS",  # replace with your own
    project_name="shared/pytorch-lightning-integration",
    experiment_name="default",  # Optional,
    params={"max_epochs": 10},  # Optional,
    tags=["pytorch-lightning", "mlp"],  # Optional,
)
trainer = Trainer(logger=neptune_logger)

NeptuneLogger 在 LightningModule 中除 __init__ 之外的任何地方都可用。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        self.logger.experiment.add_image("generated_images", some_img, 0)

## Tensorboard

要使用 TensorBoard 作为您的记录器，请执行以下操作。

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = Trainer(logger=logger)

除了 LightningModule 中的 __init__ 之外，TensorBoardLogger 可以在任何地方使用。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        self.logger.experiment.add_image("generated_images", some_img, 0)

## Test Tube

Test Tube 是一个 TensorBoard 记录器，但具有更好的文件结构。 要使用 TestTubeLogger 作为您的记录器，请执行以下操作。 首先，安装软件包：

In [ ]:
pip install test_tube

然后配置记录器并将其传递给训练器：

In [ ]:
from pytorch_lightning.loggers import TestTubeLogger

logger = TestTubeLogger("tb_logs", name="my_model")
trainer = Trainer(logger=logger)

除了 LightningModule 中的 __init__ 之外，TestTubeLogger 可以在任何地方使用。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        self.logger.experiment.add_image("generated_images", some_img, 0)

## Weights and Biases

权重和偏差是第三方记录器。 要使用 WandbLogger 作为您的记录器，请执行以下操作。 首先，安装软件包：

In [ ]:
pip install wandb

然后配置记录器并将其传递给训练器：

In [ ]:
from pytorch_lightning.loggers import WandbLogger

# instrument experiment with W&B
wandb_logger = WandbLogger(project="MNIST", log_model="all")
trainer = Trainer(logger=wandb_logger)

# log gradients and model topology
wandb_logger.watch(model)

WandbLogger 可用于 LightningModule 中除 __init__ 之外的任何地方。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        self.log({"generated_images": [wandb.Image(some_img, caption="...")]})

## 多个记录器

Lightning 支持使用多个记录器，只需向 Trainer 传递一个列表即可。

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger, TestTubeLogger

logger1 = TensorBoardLogger("tb_logs", name="my_model")
logger2 = TestTubeLogger("tb_logs", name="my_model")
trainer = Trainer(logger=[logger1, logger2])

除了 LightningModule 中的 __init__ 之外，记录器可作为列表使用。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        some_img = fake_image()
        # Option 1
        self.logger.experiment[0].add_image("generated_images", some_img, 0)
        # Option 2
        self.logger[0].experiment.add_image("generated_images", some_img, 0)

# 多 GPU 训练

Lightning 支持多种方式进行分布式训练。

## 准备你的代码

要在不更改代码的情况下在 CPU/GPU/TPU 上进行训练，我们需要养成一些好习惯 :)

## 删除 .cuda() 或 .to() 调用

删除对 .cuda() 或 .to(device) 的任何调用。

In [ ]:
# before lightning
def forward(self, x):
    x = x.cuda(0)
    layer_1.cuda(0)
    x_hat = layer_1(x)


# after lightning
def forward(self, x):
    x_hat = layer_1(x)

## 使用 type_as 和 register_buffer 初始化张量

当您需要创建新张量时，请使用 type_as。 这将使您的代码通过 Lightning 扩展到任意数量的 GPU 或 TPU。

In [ ]:
# before lightning
def forward(self, x):
    z = torch.Tensor(2, 3)
    z = z.cuda(0)


# with lightning
def forward(self, x):
    z = torch.Tensor(2, 3)
    z = z.type_as(x)

LightningModule 知道它在什么设备上。 您可以通过 `self.device` 访问参考。 有时需要将张量存储为模块属性。 但是，如果它们不是参数，即使模块移至新设备，它们仍将保留在 CPU 上。 为了防止这种情况并保持设备不可知，请使用 `register_buffer() `在模块的 `__init__` 方法中将张量注册为缓冲区。

In [ ]:
class LitModel(LightningModule):
    def __init__(self):
        ...
        self.register_buffer("sigma", torch.eye(3))
        # 您现在可以在模块的任何位置访问 self.sigma

## 移除采样器

DistributedSampler 由 Lightning 自动处理。

有关更多信息，请参阅 replace_sampler_ddp。

## 同步验证和测试日志

在分布式模式下运行时，我们必须确保验证和测试步骤日志记录调用在进程间同步。 这是通过将 sync_dist=True 添加到验证和测试步骤中的所有 self.log 调用来完成的。 这确保了每个 GPU 工作人员在跟踪模型检查点时具有相同的行为，这对于后续的下游任务（例如测试所有工作人员的最佳检查点）很重要。

请注意，如果您使用任何使用 Metrics API 的内置指标或自定义指标，则无需更新这些指标并会自动为您处理。

In [ ]:
def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    # 添加 sync_dist=True 以在所有 GPU 工作人员之间同步日志记录
    self.log("validation_loss", loss, on_step=True, on_epoch=True, sync_dist=True)


def test_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    # 添加 sync_dist=True 以在所有 GPU 工作人员之间同步日志记录
    self.log("test_loss", loss, on_step=True, on_epoch=True, sync_dist=True)

可以手动执行一些计算并将减少的结果记录在等级 0 上，如下所示：

In [ ]:
def test_step(self, batch, batch_idx):
    x, y = batch
    tensors = self(x)
    return tensors


def test_epoch_end(self, outputs):
    mean = torch.mean(self.all_gather(outputs))

# 仅在 rank 0 上登录时，不要忘记添加 ``rank_zero_only=True`` 以避免同步时出现死锁。
    if self.trainer.is_global_zero:
        self.log("my_reduced_metric", mean, rank_zero_only=True)

## 使模型可以pickle

很可能你的代码已经是pickleable的，在这种情况下不需要改变。 但是，如果您运行分布式模型并收到以下错误：

In [ ]:
self._launch(process_obj)
File "/net/software/local/python/3.6.5/lib/python3.6/multiprocessing/popen_spawn_posix.py", line 47,
in _launch reduction.dump(process_obj, fp)
File "/net/software/local/python/3.6.5/lib/python3.6/multiprocessing/reduction.py", line 60, in dump
ForkingPickler(file, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function <lambda> at 0x2b599e088ae8>:
attribute lookup <lambda> on __main__ failed

这意味着您的模型定义、转换、优化器、数据加载器或回调中的某些内容不能被pickle，并且以下代码将失败：

In [ ]:
import pickle

pickle.dump(some_object)

这是在 PyTorch 中使用多个进程进行分布式训练的限制。 要解决此问题，请找到无法腌制的代码段。 堆栈跟踪的结尾通常很有帮助。 即：在此处的堆栈跟踪示例中，代码中的某处似乎有一个无法腌制的 lambda 函数。

In [ ]:
self._launch(process_obj)
File "/net/software/local/python/3.6.5/lib/python3.6/multiprocessing/popen_spawn_posix.py", line 47,
in _launch reduction.dump(process_obj, fp)
File "/net/software/local/python/3.6.5/lib/python3.6/multiprocessing/reduction.py", line 60, in dump
ForkingPickler(file, protocol).dump(obj)
_pickle.PicklingError: Can't pickle [THIS IS THE THING TO FIND AND DELETE]:
attribute lookup <lambda> on __main__ failed

## 选择 GPU 设备

您可以使用范围、索引列表或包含以逗号分隔的 GPU id 列表的字符串来选择 GPU 设备：

In [ ]:
# DEFAULT (int) 指定每个节点使用多少 GPU
Trainer(gpus=k)

# 以上相当于
Trainer(gpus=list(range(k)))

# 指定要使用的 GPU（在集群上运行时不要使用）
Trainer(gpus=[0, 1])

# 等价于使用字符串
Trainer(gpus="0, 1")

# 要使用所有可用的 GPU，将 -1 或 '-1' 等价于 list(range(torch.cuda.device_count()))
Trainer(gpus=-1)

下表列出了可能的输入格式示例以及 Lightning 如何解释它们。 请特别注意 gpus=0、gpus=[0] 和 gpus=”0” 之间的区别。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtzwecqquhj60yv0u0dh502.jpg)

> 当 gpus 的数量指定为整数 gpus=k 时，设置 trainer 标志 auto_select_gpus=True 将自动帮助您找到未被其他进程占用的 k 个 gpu。 当 GPU 被配置为“独占模式”时，这尤其有用，这样一次只有一个进程可以访问它们。 有关更多详细信息，请参阅培训师指南。

## 选择 Torch 分布式后端

默认情况下，Lightning 在 GPU 上运行时会选择 nccl 后端而不是 gloo。 在此处查找有关 PyTorch 支持的后端的更多信息。

Lightning 公开了一个环境变量 PL_TORCH_DISTRIBUTED_BACKEND 供用户更改后端。

In [ ]:
PL_TORCH_DISTRIBUTED_BACKEND=gloo python train.py ...

## 分布式模式

* Lightning允许多种训练方式
* 数据并行（accelerator='dp'）（多GPU，1台机器）
* DistributedDataParallel (accelerator='ddp')（跨多台机器的多 GPU（基于 Python 脚本））。
* DistributedDataParallel (accelerator='ddp_spawn')（跨多台机器的多 GPU（基于 spawn））。
* DistributedDataParallel 2 (accelerator='ddp2')（机器中的DP，机器间的DDP）。
* Horovod (accelerator='horovod')（多机多gpu，运行时配置）
* TPU (tpu_cores=8|x)（tpu 或 TPU pod）

> 如果您在未设置模式的情况下请求多个 GPU 或节点，则会自动使用 DDP Spawn。

要更深入地了解 Lightning 正在做什么，请随时阅读本指南。

### 数据并行

DataParallel (DP) 在 k 个 GPU 上拆分一个批次。 也就是说，如果你有一批 32 个，使用 DP 和 2 个 gpu，每个 GPU 将处理 16 个样本，之后根节点将聚合结果。

> PyTorch 和 Lightning 不鼓励使用 DP。 状态不在 DataParallel 包装器创建的副本上维护，如果您在 forward() 或 *_step() 方法中将状态分配给模块，您可能会看到错误或不当行为。 出于同样的原因，我们不能完全支持使用 DP 进行手动优化。 使用更稳定且速度至少快 3 倍的 DDP。

> DP 仅支持散布和收集原始张量集合，如列表、字典等。因此 transfer_batch_to_device() 钩子不适用于此模式，如果您已覆盖它，则不会调用它。

In [ ]:
# train on 2 GPUs (using DP mode)
trainer = Trainer(gpus=2, accelerator="dp")

### 分布式数据并行

DistributedDataParallel (DDP) 的工作原理如下：
* 每个节点上的每个 GPU 都有自己的进程。
* 每个 GPU 都可以查看整个数据集的一个子集。 它只会看到那个子集。
* 每个进程都会初始化模型。
* 每个进程并行执行完整的向前和向后传递。
* 梯度在所有过程中同步和平均。
* 每个进程都会更新其优化器。

In [ ]:
# train on 8 GPUs (same machine (ie: node))
trainer = Trainer(gpus=8, accelerator="ddp")

# train on 32 GPUs (4 nodes)
trainer = Trainer(gpus=8, accelerator="ddp", num_nodes=4)

我们以这种方式使用 DDP，因为 ddp_spawn 有一些限制（由于 Python 和 PyTorch）：
1. 由于 .spawn() 在子流程中训练模型，因此主流程上的模型不会更新。
2. Dataloader(num_workers=N)，其中 N 很大，使用 DDP 进行训练会遇到瓶颈……即：它会非常慢或根本无法工作。 这是 PyTorch 的限制。
3. 强制一切都可以腌制。

有些情况下无法使用 DDP。 例子是：
1. Jupyter Notebook、Google COLAB、Kaggle 等
2. 你有一个没有根包的嵌套脚本

在这些情况下，您应该改用 dp 或 ddp_spawn。

### 分布式数据并行 2

在某些情况下，在同一台机器上使用所有批次而不是子集是有利的。 例如，您可能想要计算一个 NCE 损失，其中有更多的负样本是值得的。

在这种情况下，我们可以使用 DDP2，它的行为类似于机器中的 DP 和跨节点的 DDP。 DDP2 执行以下操作：
* 将数据的一个子集复制到每个节点。
* 在每个节点上初始化一个模型。
* 使用 DP 运行向前和向后传递。
* 跨节点同步梯度。
* 应用优化器更新。

In [ ]:
# train on 32 GPUs (4 nodes)
trainer = Trainer(gpus=8, accelerator="ddp2", num_nodes=4)

### 分布式数据并行生成

ddp_spawn 与 ddp 完全一样，只是它使用 .spawn 来启动训练过程。

> 强烈建议使用 DDP 来提高速度和性能。

In [ ]:
mp.spawn(self.ddp_train, nprocs=self.num_processes, args=(model,))

如果您的脚本不支持从命令行调用（即：它嵌套在没有根项目模块的情况下），您可以使用以下方法：

In [ ]:
# train on 8 GPUs (same machine (ie: node))
trainer = Trainer(gpus=8, accelerator="ddp_spawn")

我们强烈反对这种使用，因为它有局限性（由于 Python 和 PyTorch）：
* 您传入的模型不会更新。 请保存检查点并从那里恢复。
* 设置 Dataloader(num_workers=0) 否则会成为训练瓶颈。

ddp 比 ddp_spawn 快得多。 我们推荐你
1. 使用 setup.py 为您的项目安装顶级模块

In [ ]:
# setup.py
#!/usr/bin/env python

from setuptools import setup, find_packages

setup(
    name="src",
    version="0.0.1",
    description="Describe Your Cool Project",
    author="",
    author_email="",
    url="https://github.com/YourSeed",  # REPLACE WITH YOUR OWN GITHUB PROJECT LINK
    install_requires=["pytorch-lightning"],
    packages=find_packages(),
)

2. 像这样设置你的项目：

In [ ]:
/project
    /src
        some_file.py
        /or_a_folder
    setup.py

3. 作为根级包安装

In [ ]:
cd /project
pip install -e .

然后你可以在任何地方调用你的脚本

In [ ]:
cd /project/src
python some_file.py --accelerator 'ddp' --gpus 8

### Horovod

Horovod 允许将相同的训练脚本用于单 GPU、多 GPU 和多节点训练。

与分布式数据并行一样，Horovod 中的每个进程都在具有固定数据子集的单个 GPU 上运行。 在反向传递期间，梯度在所有 GPU 上并行平均，然后在开始下一步之前同步应用。

工作进程的数量由驱动程序应用程序（horovodrun 或 mpirun）配置。 在训练脚本中，Horovod 将检测环境中的工人数量，并自动调整学习率以补偿增加的总批次大小。

可以在训练脚本中配置 Horovod 以运行任意数量的 GPU/进程，如下所示：

In [ ]:
# train Horovod on GPU (number of GPUs / machines provided on command-line)
trainer = Trainer(accelerator="horovod", gpus=1)

# train Horovod on CPU (number of processes / machines provided on command-line)
trainer = Trainer(accelerator="horovod")

开始训练作业时，驱动程序应用程序将用于指定工作进程的总数：

In [ ]:
# run training with 4 GPUs on a single machine
horovodrun -np 4 python train.py

# run training with 8 GPUs on two machines (4 GPUs each)
horovodrun -np 8 -H hostname1:4,hostname2:4 python train.py

### DP/DDP2 警告

在 DP 和 DDP2 中，机器中的每个 GPU 都会看到一批的一部分。 DP和ddp2大致做了以下几件事：

In [ ]:
def distributed_forward(batch, model):
    batch = torch.Tensor(32, 8)
    gpu_0_batch = batch[:8]
    gpu_1_batch = batch[8:16]
    gpu_2_batch = batch[16:24]
    gpu_3_batch = batch[24:]

    y_0 = model_copy_gpu_0(gpu_0_batch)
    y_1 = model_copy_gpu_1(gpu_1_batch)
    y_2 = model_copy_gpu_2(gpu_2_batch)
    y_3 = model_copy_gpu_3(gpu_3_batch)

    return [y_0, y_1, y_2, y_3]

因此，当 Lightning 调用 training_step、validation_step、test_step 中的任何一个时，您将只对其中一个进行操作。

In [ ]:
# the batch here is a portion of the FULL batch
def training_step(self, batch, batch_idx):
    y_0 = batch

对于大多数指标，这并不重要。 但是，如果您想使用所有批处理部分向计算图中添加一些内容（如 softmax），您可以使用 training_step_end 步骤。

In [ ]:
def training_step_end(self, outputs):
    # only use when  on dp
    outputs = torch.cat(outputs, dim=1)
    softmax = softmax(outputs, dim=1)
    out = softmax.mean()
    return out

在伪代码中，完整的序列是：

In [ ]:
# get data
batch = next(dataloader)

# copy model and data to each gpu
batch_splits = split_batch(batch, num_gpus)
models = copy_model_to_gpus(model)

# in parallel, operate on each batch chunk
all_results = []
for gpu_num in gpus:
    batch_split = batch_splits[gpu_num]
    gpu_model = models[gpu_num]
    out = gpu_model(batch_split)
    all_results.append(out)

# use the full batch for something like softmax
full_out = model.training_step_end(all_results)

为了说明为什么需要这样做，让我们看一下 DataParallel

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(batch)

    # on dp or ddp2 if we did softmax now it would be wrong
    # because batch is actually a piece of the full batch
    return y_hat


def training_step_end(self, batch_parts_outputs):
    # batch_parts_outputs has outputs of each part of the batch

    # do softmax here
    outputs = torch.cat(outputs, dim=1)
    softmax = softmax(outputs, dim=1)
    out = softmax.mean()

    return out

如果定义了 training_step_end，它将被调用而不管 TPU、DP、DDP 等等……这意味着不管后端如何，它的行为都是一样的。

使用 DP 时，验证和测试步骤具有相同的选项。

In [ ]:
def validation_step_end(self, batch_parts_outputs):
    ...


def test_step_end(self, batch_parts_outputs):
    ...

### 分布式和 16 位精度

由于 Apex 和 DataParallel 的问题（PyTorch 和 NVIDIA 问题），Lightning 不允许 16 位和 DP 训练。 我们试图让它发挥作用，但这对他们来说是一个问题。

以下是我们支持的可能配置。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtzxyap2msj61940swjst02.jpg)

### 实施您自己的分布式 (DDP) 训练

如果您需要自己的方式来初始化 PyTorch DDP，您可以覆盖 pytorch_lightning.plugins.training_type.ddp.DDPPlugin.init_ddp_connection()。

如果您还需要使用自己的 DDP 实现，请覆盖 pytorch_lightning.plugins.training_type.ddp.DDPPlugin.configure_ddp()。

### 批量大小

使用分布式训练时，请确保根据您的有效批量大小修改您的学习率。

假设您的数据加载器中的批次大小为 7。

In [ ]:
class LitModel(LightningModule):
    def train_dataloader(self):
        return Dataset(..., batch_size=7)

在 (DDP, Horovod) 中，您的有效批量大小将为 7 * gpus * num_nodes。

In [ ]:
# effective batch size = 7 * 8
Trainer(gpus=8, accelerator="ddp|horovod")

# effective batch size = 7 * 8 * 10
Trainer(gpus=8, num_nodes=10, accelerator="ddp|horovod")

在 DDP2 中，您的有效批量大小将为 7 * num_nodes。 原因是使用 DDP2 时，节点上的所有 GPU 都可以看到完整批次。

In [ ]:
# effective batch size = 7
Trainer(gpus=8, accelerator="ddp2")

# effective batch size = 7 * 10
Trainer(gpus=8, num_nodes=10, accelerator="ddp2")

> 大批量实际上对收敛非常不利。 查看：准确、大型 Minibatch SGD：1 小时内训练 ImageNet

### Torch分布式弹性

Lightning 支持使用 Torch Distributed Elastic 来实现容错和弹性的分布式作业调度。 要使用它，请指定“ddp”或“ddp2”后端以及要在训练器中使用的 GPU 数量。

In [ ]:
Trainer(gpus=8, accelerator="ddp")

要启动容错作业，请在所有节点上运行以下命令。

In [ ]:
python -m torch.distributed.run
        --nnodes=NUM_NODES
        --nproc_per_node=TRAINERS_PER_NODE
        --rdzv_id=JOB_ID
        --rdzv_backend=c10d
        --rdzv_endpoint=HOST_NODE_ADDR
        YOUR_LIGHTNING_TRAINING_SCRIPT.py (--arg1 ... train script args...)

要启动弹性作业，请在至少 MIN_SIZE 节点和最多 MAX_SIZE 节点上运行以下命令。

In [ ]:
python -m torch.distributed.run
        --nnodes=MIN_SIZE:MAX_SIZE
        --nproc_per_node=TRAINERS_PER_NODE
        --rdzv_id=JOB_ID
        --rdzv_backend=c10d
        --rdzv_endpoint=HOST_NODE_ADDR
        YOUR_LIGHTNING_TRAINING_SCRIPT.py (--arg1 ... train script args...)

### Jupyter Notebook

不幸的是，jupyter 笔记本不支持任何 ddp_。 请为多个 GPU 使用 dp。 这是一个已知的 Jupyter 问题。 如果您想尝试添加此支持，请随时提交 PR！

### Pickle错误

多 GPU 训练有时需要对您的模型进行腌制。 如果您遇到酸洗问题，请尝试以下操作来找出问题所在

In [ ]:
import pickle

model = YourModel()
pickle.dumps(model)

但是，如果您使用 ddp，则酸洗要求不存在，您应该没问题。 如果您使用 ddp_spawn，则pickling 要求仍然存在。 这是 Python 的一个限制。

# 模型并行 GPU 训练

在训练大型模型、拟合更大的批量大小或尝试使用多 GPU 计算增加吞吐量时，Lightning 提供了高级优化的分布式训练插件来支持这些情况并显着改善内存使用。

在许多情况下，这些插件是模型并行的某种风格，但是我们只介绍高级概念以帮助您入门。 有关模型并行性的更多信息，请参阅 FairScale 文档。

请注意，一些极端的内存节省配置会影响训练速度。 在大多数情况下，这种速度/内存权衡是可以调整的。

其中一些内存高效插件依赖于卸载到其他形式的内存，例如 CPU RAM 或 NVMe。 这意味着您甚至可以使用 DeepSpeed ZeRO Stage 3 Offload 等插件在单个 GPU 上看到内存优势。

## 选择高级分布式 GPU 插件

如果您想坚持使用 PyTorch DDP，请参阅 DDP 优化。

与 PyTorch 的 DistributedDataParallel (DDP) 不同，最大可训练模型大小和批量大小不会随 GPU 数量而变化，内存优化插件可以在使用更多 GPU 时容纳更大的模型和更大的批次。 这意味着随着 GPU 数量的增加，您可以达到想要训练的模型参数数量。

选择插件时有很多考虑因素，如下所述。 此外，请在此处查看使用 minGPT 的各种插件基准测试的可视化。

## 预训练与微调

在微调时，与预训练模型相比，我们经常使用更少的数据。 这在选择分布式插件时很重要，因为它通常用于预训练，因为我们是计算受限的。 这意味着我们不能像微调那样牺牲吞吐量，因为在微调中，数据需求更小。

总体上：
* 在微调模型时，使用高级内存高效插件，例如 DeepSpeed ZeRO Stage 3 或 DeepSpeed ZeRO Stage 3 Offload，如果您的计算能力有限，您可以微调更大的模型
* 在预训练模型时，使用更简单的优化，例如分片训练、DeepSpeed ZeRO Stage 2 或完全分片训练，扩展 GPU 数量以达到更大的参数大小
* 对于微调和预训练，请使用 DeepSpeed 激活检查点或 FairScale 激活检查点，因为吞吐量下降并不显着

例如，当使用 128 个 GPU 时，您可以使用 DeepSpeed ZeRO Stage 2 预训练 10 到 200 亿个大型参数模型，而无需使用更高级的优化多 GPU 插件来降低性能。

但是为了对模型进行微调，您可以在单个 GPU 上使用 DeepSpeed ZeRO Stage 3 Offload 达到 10 到 200 亿个参数模型。 这确实会对吞吐量造成重大影响，需要相应地进行权衡。

## 什么时候不应该使用优化的分布式插件？

当模型大小相当大时，分片技术会有所帮助； 大约 500M+ 的参数是我们看到好处的地方。 但是，如果您的模型很小（大约 80M 参数的 ResNet50），最好坚持普通的分布式训练，除非您使用异常大的批量或输入。

## 分片训练

FairScale 提供的优化器分片训练的闪电集成。该技术可以在 DeepSpeed ZeRO 和 ZeRO-2 中找到，但是实现是从头开始构建的，与 pytorch 兼容和独立。分片训练允许您保持 GPU 扩展效率，同时大幅减少内存开销。简而言之，期望接近正常的线性缩放（如果您的网络允许），并在训练大型模型时显着减少内存使用。

分片训练仍然在幕后使用数据并行训练，除了优化器状态和梯度跨 GPU 分片。这意味着每个 GPU 的内存开销较低，因为每个 GPU 只需维护优化器状态和梯度的分区。

好处因模型和参数大小而异，但我们记录了每个 GPU 的内存减少高达 63%，使我们能够将模型大小增加一倍。由于有效的通信，多 GPU 设置中的这些好处几乎是免费的，并且吞吐量可以通过多节点设置很好地扩展。

强烈建议在内存有限的多 GPU 环境中使用分片训练，或者训练更大的模型是有益的（500M+ 参数模型）。技术说明：随着批量大小的扩展，存储向后传递的激活成为训练的瓶颈。因此，分片优化器状态和梯度的影响较小。使用 FairScale Activation Checkpointing 以牺牲一些吞吐量为代价获得更多好处。

要使用分片训练，您需要首先使用以下命令安装 FairScale。

In [ ]:
pip install fairscale

In [ ]:
# train using Sharded DDP
trainer = Trainer(plugins="ddp_sharded")

通过添加额外的 --plugins ddp_sharded 标志，分片训练可以适用于所有 DDP 变体。

在内部，我们重新初始化您的优化器并将它们分片到您的机器和流程中。 我们使用分布式 PyTorch 处理所有通信，因此不需要更改代码。

## 完全分片训练

> 完全分片训练处于测试阶段，API 可能会发生变化。 如果您遇到任何问题，请创建一个问题。

跨数据并行工作者的完全分片优化器状态、梯度和参数。 这允许您将更大的模型安装到多个 GPU 到内存中。

完全分片训练减轻了使用某种形式的管道并行性在特定设备上平衡层的需要，并以最少的努力优化分布式通信。

### 分片参数达到 10+ 十亿个参数

为了达到更大的参数大小并提高内存效率，我们必须对参数进行分片。 有多种方法可以实现这一点。

> 目前全分片训练依赖于用户在 LightningModule 中使用全分片包装模型。 这意味着您必须在 LightningModule 中创建一个被视为 torch.nn.Module 的模型。 这是完全分片训练的一个限制，将在未来解决。

### 启用模块分片以实现最大内存效率

要激活参数分片，您必须使用提供的 wrap 或 auto_wrap 函数包装模型，如下所述。在 Lightning 内部，我们围绕 configure_sharded_model 函数启用上下文管理器，以确保正确传递 wrap 和 auto_wrap 参数。

当不使用完全分片时，这些包装函数是无操作的。这意味着一旦进行了更改，就无需删除其他插件的更改。

auto_wrap 将使用嵌套的全分片包装器递归地将 torch.nn.Module 包装在 LightningModule 中，这表明我们希望将这些模块跨数据并行设备进行分区，并在不需要时丢弃全部权重（此处的信息）。

auto_wrap 可以根据模型的复杂程度获得不同程度的成功。 Auto Wrap 不支持具有共享参数的模型。

wrap 将使用来自 Lightning 上下文管理器的正确参数简单地使用完全分片并行类包装模块。

下面是使用 wrap 和 auto_wrap 创建模型的示例。

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from fairscale.nn import checkpoint_wrapper, auto_wrap, wrap


class MyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(32, 32)
        self.block = nn.Sequential(nn.Linear(32, 32), nn.ReLU())
        self.final_block = nn.Sequential(nn.Linear(32, 32), nn.ReLU())

    def configure_sharded_model(self):
        # modules are sharded across processes
        # as soon as they are wrapped with ``wrap`` or ``auto_wrap``.
        # During the forward/backward passes, weights get synced across processes
        # and de-allocated once computation is complete, saving memory.

        # Wraps the layer in a Fully Sharded Wrapper automatically
        linear_layer = wrap(self.linear_layer)

        # Wraps the module recursively
        # based on a minimum number of parameters (default 100M parameters)
        block = auto_wrap(self.block)

        # For best memory efficiency,
        # add FairScale activation checkpointing
        final_block = auto_wrap(checkpoint_wrapper(self.final_block))
        self.model = nn.Sequential(linear_layer, nn.ReLU(), block, final_block)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters())


model = MyModel()
trainer = Trainer(gpus=4, plugins="fsdp", precision=16)
trainer.fit(model)

trainer.test()
trainer.predict()

## FairScale 激活检查点

一旦在前向传递期间不需要激活，激活检查点就会从内存中释放激活。 然后根据需要为向后传递重新计算它们。 当您有产生大量激活的中间层时，激活检查点非常有用。

与 PyTorch 实现不同，FairScales 的检查点包装器还可以正确处理批规范层，确保由于多次前向传递而正确跟踪统计数据。

这在训练较大模型时可以节省内存，但需要包装您想要使用激活检查点的模块。 浏览此处获取更多信息。

> 确保不要使用激活检查点来包装整个模型。 这不是激活检查点的预期用途，并且会导致失败，如本讨论中所见。

In [ ]:
from pytorch_lightning import Trainer
from fairscale.nn import checkpoint_wrapper


class MyModel(pl.LightningModule):
    def __init__(self):
        # Wrap layers using checkpoint_wrapper
        self.block_1 = checkpoint_wrapper(nn.Sequential(nn.Linear(32, 32), nn.ReLU()))
        self.block_2 = nn.Linear(32, 2)

## DeepSpeed

DeepSpeed 插件处于测试阶段，API 可能会发生变化。 如果您遇到任何问题，请创建一个问题。

DeepSpeed 是一个深度学习训练优化库，提供大规模训练海量十亿参数模型的手段。 使用 DeepSpeed 插件，我们能够训练 100 亿及以上参数的模型大小，在此基准测试和 DeepSpeed 文档中有很多有用的信息。 DeepSpeed 还提供较低级别的训练优化和高效的优化器，例如 1-bit Adam。 我们建议在速度和内存优化很重要的环境中使用 DeepSpeed（例如训练大型 10 亿参数模型）。

以下是 DeepSpeed 的所有配置的摘要。
* DeepSpeed ZeRO Stage 1 - 分片优化器状态，​​在提供内存改进的同时保持与 DDP 相同的速度
* DeepSpeed ZeRO Stage 2 - 分片优化器的状态和梯度，与 DDP 保持同等速度，同时提供更多内存改进
* DeepSpeed ZeRO Stage 2 Offload - 将优化器状态和梯度卸载到 CPU。增加分布式通信量和 GPU-CPU 设备传输，但提供显着的内存改进
* DeepSpeed ZeRO Stage 3 - 分片优化器状态、梯度、参数和可选的激活。增加分布式通信量，但提供更多内存改进
* DeepSpeed ZeRO Stage 3 Offload - 将优化器状态、梯度、参数和可选的激活卸载到 CPU。增加分布式通信量和 GPU-CPU 设备传输，但更显着的内存改进。
* DeepSpeed 激活检查点 - 前向传递后免费激活。增加计算，但为所有阶段提供内存改进。

要使用 DeepSpeed，您首先需要使用以下命令安装 DeepSpeed。

In [ ]:
pip install deepspeed

如果您在安装或稍后的培训中遇到问题，请确保您安装的 pytorch 的 CUDA 版本与您本地安装的 CUDA 匹配（您可以通过运行 nvcc --version 查看已识别出哪个版本）。

> DeepSpeed 目前仅支持训练循环内的单个优化器、单个调度器。
> 
> 保存检查点时，我们依赖 DeepSpeed，它保存包含模型和各种组件的目录。

### DeepSpeed 零阶段 1

DeepSpeed ZeRO Stage 1 在 GPU 上划分优化器状态（Stage 1）以减少内存。

建议跳过第 1 阶段并使用第 2 阶段，它具有更大的内存改进并且仍然保持高效。 第 1 阶段对于与某些优化（例如 Torch ORT）配对很有用。

In [ ]:
from pytorch_lightning import Trainer

model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_1", precision=16)
trainer.fit(model)

### DeepSpeed 零阶段 2

DeepSpeed ZeRO Stage 2 在 GPU 上划分优化器状态（阶段 1）和梯度（阶段 2）以减少内存。 在大多数情况下，这更有效或与 DDP 持平，主要是由于 DeepSpeed 团队编写的优化自定义通信。 因此，在单个 GPU 上也可以看到好处。 请注意，默认存储桶大小分配了大约 3.6GB 的 VRAM 用于分布式通信，这可以在实例化下面几节中描述的插件时进行调整。

In [ ]:
from pytorch_lightning import Trainer

model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_2", precision=16)
trainer.fit(model)

In [ ]:
python train.py --plugins deepspeed_stage_2 --precision 16 --gpus 4

### DeepSpeed ZeRO Stage 2 卸载

下面我们展示了一个运行 ZeRO-Offload 的例子。 ZeRO-Offload 利用主机 CPU 来卸载优化器内存/计算，从而减少整体内存消耗。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_2_offload", precision=16)
trainer.fit(model)

这也可以通过命令行使用 Pytorch Lightning 脚本来完成：

In [ ]:
python train.py --plugins deepspeed_stage_2_offload --precision 16 --gpus 4

您还可以通过插件修改 ZeRO-Offload 参数，如下所示。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

model = MyModel()
trainer = Trainer(
    gpus=4, plugins=DeepSpeedPlugin(cpu_offload=True, allgather_bucket_size=5e8, reduce_bucket_size=5e8), precision=16
)
trainer.fit(model)

> 我们建议调整 allgather_bucket_size 参数和 reduce_bucket_size 参数以根据您的模型大小找到最佳参数。 这些控制我们在减少梯度/收集更新参数时限制模型使用的缓冲区大小。 较小的值会导致更少的内存，但会影响速度。
> 
> DeepSpeed 分配了乘以 4.5 倍的缩减缓冲区大小，因此在调整参数时要考虑到这一点。
> 
> 该插件设置了一个合理的默认值 2e8，它应该适用于大多数低 VRAM GPU（小于 7GB），分配大约 3.6GB 的 VRAM 作为缓冲区。 更高的 VRAM GPU 应该针对 5e8 左右的值。

为了获得更多的速度优势，DeepSpeed 提供了一个优化的 ADAM CPU 版本，称为 DeepSpeedCPUAdam 来运行卸载计算，这比标准的 PyTorch 实现更快。

In [ ]:
import pytorch_lightning
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin
from deepspeed.ops.adam import DeepSpeedCPUAdam


class MyModel(pl.LightningModule):
    ...

    def configure_optimizers(self):
        # DeepSpeedCPUAdam provides 5x to 7x speedup over torch.optim.adam(w)
        return DeepSpeedCPUAdam(self.parameters())


model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_2_offload", precision=16)
trainer.fit(model)

### DeepSpeed 零级第 3 阶段

DeepSpeed ZeRO Stage 3 对优化器状态、梯度和模型参数（也可选择激活）进行分片。分片模型参数和激活伴随着分布式通信的增加，但允许您将模型从一个 GPU 大规模扩展到多个 GPU。 DeepSpeed 团队报告了在单个 GPU 上微调具有超过 40B 参数和在 512 个 GPU 上超过 2 万亿参数的模型的能力。有关更多信息，我们建议查看 DeepSpeed ZeRO-3 Offload 文档。

我们已经为所有这些功能运行了基准测试，并给出了一个简单的示例，说明所有这些功能如何在 Lightning 中工作，您可以在 minGPT 上看到。

要达到最高的内存效率或模型大小，您必须：
1. 使用带有第 3 阶段参数的 DeepSpeed 插件
2. 使用 CPU Offloading 将权重卸载到 CPU，并有合理数量的 CPU RAM 卸载到
3. 使用 DeepSpeed 激活检查点对激活进行分片

下面我们将描述如何使所有这些都看到好处。通过所有这些改进，我们在 8 个 GPU 上训练 GPT 模型的参数达到了 450 亿，可用 CPU RAM 约为 1TB。

另请查看我们的 DeepSpeed ZeRO Stage 3 Tips，其中包含配置您自己的模型时的许多有用信息。

> 使用 DeepSpeed 和 Stage 3 保存模型时，模型状态和优化器状态将保存在单独的分片状态中（基于世界大小）。 请参阅为 DeepSpeed ZeRO Stage 3 整理单个文件检查点以获取单个检查点文件。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin
from deepspeed.ops.adam import FusedAdam


class MyModel(pl.LightningModule):
    ...

    def configure_optimizers(self):
        return FusedAdam(self.parameters())


model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_3", precision=16)
trainer.fit(model)

trainer.test()
trainer.predict()

一旦模型经过训练，您还可以使用 Lightning Trainer 使用 DeepSpeed 运行预测或评估。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin


class MyModel(pl.LightningModule):
    ...


model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_3", precision=16)
trainer.test(ckpt_path="my_saved_deepspeed_checkpoint.ckpt")

### 立即分片模型以减少初始化时间/内存

在实例化非常大的模型时，有时需要立即对模型层进行分片。

如果层可能不适合一台机器的 CPU 或 GPU 内存，但一旦在多台机器上分片，就会适合这种情况。 我们公开了一个钩子，钩子内初始化的层将在每层的基础上立即分片，允许您立即分片模型。

这减少了初始化非常大的模型所需的时间，并确保我们在实例化较大的模型时不会耗尽内存。 有关更多信息，您可以参考用于构建大规模模型的 DeepSpeed 文档。

In [ ]:
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin
from deepspeed.ops.adam import FusedAdam


class MyModel(pl.LightningModule):
    ...

    def configure_sharded_model(self):
        # Created within sharded model context, modules are instantly sharded across processes
        # as soon as they are made.
        self.block = nn.Sequential(nn.Linear(32, 32), nn.ReLU())

    def configure_optimizers(self):
        return FusedAdam(self.parameters())


model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_3", precision=16)
trainer.fit(model)

trainer.test()
trainer.predict()

### DeepSpeed ZeRO Stage 3 卸载

DeepSpeed ZeRO Stage 3 卸载优化器状态，梯度到主机 CPU 以减少内存使用，就像 ZeRO Stage 2 一样，但另外还允许您卸载参数以节省更多内存。

> 使用 DeepSpeed 和 Stage 3 保存模型时，模型状态和优化器状态将保存在单独的分片状态中（基于世界大小）。 请参阅为 DeepSpeed ZeRO Stage 3 整理单个文件检查点以获取单个检查点文件。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

# Enable CPU Offloading
model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_3_offload", precision=16)
trainer.fit(model)

# Enable CPU Offloading, and offload parameters to CPU
model = MyModel()
trainer = Trainer(
    gpus=4,
    plugins=DeepSpeedPlugin(
        stage=3,
        offload_optimizer=True,
        offload_parameters=True,
    ),
    precision=16,
)
trainer.fit(model)

### DeepSpeed Infinity（NVMe 卸载）

此外，利用 NVMe 中的大内存空间，DeepSpeed 支持为更大的模型卸载到 NVMe 驱动器。 DeepSpeed 报告了在一台 8 GPU 机器上使用 NVMe 卸载微调 1 万亿以上参数的能力。 下面显示了如何启用此功能，假设 NVMe 驱动器安装在名为 /local_nvme 的目录中。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

# Enable CPU Offloading
model = MyModel()
trainer = Trainer(gpus=4, plugins="deepspeed_stage_3_offload", precision=16)
trainer.fit(model)

# Enable CPU Offloading, and offload parameters to CPU
model = MyModel()
trainer = Trainer(
    gpus=4,
    plugins=DeepSpeedPlugin(
        stage=3,
        offload_optimizer=True,
        offload_parameters=True,
        remote_device="nvme",
        offload_params_device="nvme",
        offload_optimizer_device="nvme",
        nvme_path="/local_nvme",
    ),
    precision=16,
)
trainer.fit(model)

卸载到 NVMe 时，您可能会注意到速度很慢。 有些参数需要根据您使用的驱动器进行调整。 运行 aio_bench_perf_sweep.py 脚本可以帮助您找到最佳参数。 有关如何解析信息的更多信息，请参阅问题。

### DeepSpeed 激活检查点

一旦在前向传递期间不需要激活，激活检查点就会从内存中释放激活。 然后根据需要为向后传递重新计算它们。

当您有产生大量激活的中间层时，激活检查点非常有用。

这在训练较大模型时可以节省内存，但是需要使用检查点函数来运行模块，如下所示。

> 确保不要使用激活检查点来包装整个模型。 这不是激活检查点的预期用途，并且会导致失败，如本讨论中所见。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin
import deepspeed


class MyModel(LightningModule):
    ...

    def __init__(self):
        super().__init__()
        self.block_1 = nn.Sequential(nn.Linear(32, 32), nn.ReLU())
        self.block_2 = torch.nn.Linear(32, 2)

    def forward(self, x):
        # Use the DeepSpeed checkpointing function instead of calling the module directly
        # checkpointing self.layer_h means the activations are deleted after use,
        # and re-calculated during the backward passes
        x = torch.utils.checkpoint.checkpoint(self.block_1, x)
        return self.block_2(x)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin
import deepspeed


class MyModel(pl.LightningModule):
    ...

    def configure_sharded_model(self):
        self.block_1 = nn.Sequential(nn.Linear(32, 32), nn.ReLU())
        self.block_2 = torch.nn.Linear(32, 2)

    def forward(self, x):
        # Use the DeepSpeed checkpointing function instead of calling the module directly
        x = deepspeed.checkpointing.checkpoint(self.block_1, x)
        return self.block_2(x)


model = MyModel()


trainer = Trainer(gpus=4, plugins="deepspeed_stage_3_offload", precision=16)

# Enable CPU Activation Checkpointing
trainer = Trainer(
    gpus=4,
    plugins=DeepSpeedPlugin(
        stage=3,
        cpu_offload=True,  # Enable CPU Offloading
        cpu_checkpointing=True,  # (Optional) offload activations to CPU
    ),
    precision=16,
)
trainer.fit(model)

### DeepSpeed ZeRO 第 3 阶段提示

以下是使用 Lightning 设置 DeepSpeed ZeRO Stage 3 时的一些有用信息。
* 如果您使用 Adam 或 AdamW，请确保使用 FusedAdam 或 DeepSpeedCPUAdam（用于 CPU 卸载）而不是默认的 Torch 优化器，因为它们具有很大的速度优势
* 将您的 GPU/CPU 内存视为一个大池。 在某些情况下，您可能不想卸载某些东西（如激活）以提供更多空间来卸载模型参数
* 当卸载到 CPU 时，确保增加批量大小，因为 GPU 内存将被释放
* 我们还支持分片检查点。 通过将 save_full_weights=False 传递给 DeepSpeedPlugin，我们将保存模型的碎片，从而允许您保存非常大的模型。 但是，要加载模型并运行测试/验证/预测，您必须使用 Trainer 对象。

### 整理 DeepSpeed ZeRO Stage 3 的单个文件检查点

使用 ZeRO Stage 3 进行训练后，您会注意到检查点是分片模型和优化器状态的目录。 如果您想从检查点目录中整理单个文件，请使用以下命令，该命令在整理文件时额外处理所有闪电状态。

In [ ]:
from pytorch_lightning.utilities.deepspeed import convert_zero_checkpoint_to_fp32_state_dict

# lightning deepspeed has saved a directory instead of a file
save_path = "lightning_logs/version_0/checkpoints/epoch=0-step=0.ckpt/"
output_path = "lightning_model.pt"
convert_zero_checkpoint_to_fp32_state_dict(save_path, output_path)

> 这个单一文件检查点不包括优化器/lr-scheduler 状态。 这意味着我们无法通过 resume_from_checkpoint Trainer 参数恢复训练。 如果需要，请确保保留分片检查点目录。

### 自定义 DeepSpeed 配置

在某些情况下，您可能希望定义自己的 DeepSpeed 配置，以访问定义的所有参数。 我们已经公开了大部分重要参数，但是，可能需要启用调试参数。 此外，DeepSpeed 允许使用在受支持的配置文件中定义的自定义 DeepSpeed 优化器和调度器。

> 传递配置对象时，将忽略所有插件默认参数。 所有兼容的参数都可以在 DeepSpeed 文档中看到。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

deepspeed_config = {
    "zero_allow_untested_optimizer": True,
    "optimizer": {
        "type": "OneBitAdam",
        "params": {
            "lr": 3e-5,
            "betas": [0.998, 0.999],
            "eps": 1e-5,
            "weight_decay": 1e-9,
            "cuda_aware": True,
        },
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "last_batch_iteration": -1,
            "warmup_min_lr": 0,
            "warmup_max_lr": 3e-5,
            "warmup_num_steps": 100,
        },
    },
    "zero_optimization": {
        "stage": 2,  # Enable Stage 2 ZeRO (Optimizer/Gradient state partitioning)
        "cpu_offload": True,  # Enable Offloading optimizer state/calculation to the host CPU
        "contiguous_gradients": True,  # Reduce gradient fragmentation.
        "overlap_comm": True,  # Overlap reduce/backward operation of gradients for speed.
        "allgather_bucket_size": 2e8,  # Number of elements to all gather at once.
        "reduce_bucket_size": 2e8,  # Number of elements we reduce/allreduce at once.
    },
}

model = MyModel()
trainer = Trainer(gpus=4, plugins=DeepSpeedPlugin(deepspeed_config), precision=16)
trainer.fit(model)

我们支持将配置作为 json 格式的文件：

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DeepSpeedPlugin

model = MyModel()
trainer = Trainer(gpus=4, plugins=DeepSpeedPlugin("/path/to/deepspeed_config.json"), precision=16)
trainer.fit(model)

您还可以通过 PyTorch Lightning 脚本使用环境变量：

In [ ]:
PL_DEEPSPEED_CONFIG_PATH=/path/to/deepspeed_config.json python train.py --plugins deepspeed

## DDP 优化

### 渐变作为桶视图

在 DDPPlugin 中启用 gradient_as_bucket_view=True 将使梯度视图指向 allreduce 通信桶的不同偏移量。 有关更多信息，请参阅 DistributedDataParallel。

这可以减少峰值内存使用量和吞吐量，因为节省的内存将等于总梯度内存 + 消除将梯度复制到 allreduce 通信桶的需要。

> 当gradient_as_bucket_view=True 时，您不能在渐变上调用 detach_() 。 如果遇到此类错误，请参考 torch/optim/optimizer.py 中的 zero_grad() 函数作为解决方案（来源）进行修复。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DDPPlugin

model = MyModel()
trainer = Trainer(gpus=4, plugins=DDPPlugin(gradient_as_bucket_view=True))
trainer.fit(model)

### DDP 通信挂钩

DDP 通信钩子是一个接口，用于控制梯度如何在工作人员之间进行通信，覆盖 DistributedDataParallel 中的标准 allreduce。 这允许您在使用多个节点时启用性能改进通信挂钩。

> DDP 通信钩子需要 pytorch 版本至少为 1.8.0

启用 FP16 Compress Hook 以提高多节点吞吐量：

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DDPPlugin
from torch.distributed.algorithms.ddp_comm_hooks import (
    default_hooks as default,
    powerSGD_hook as powerSGD,
)

model = MyModel()
trainer = Trainer(gpus=4, plugins=DDPPlugin(ddp_comm_hook=default.fp16_compress_hook))
trainer.fit(model)

启用 PowerSGD 以提高多节点吞吐量：

> PowerSGD 通常需要与模型梯度相同大小的额外内存以启用错误反馈，这可以补偿有偏差的压缩通信并提高准确性（来源）。

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DDPPlugin
from torch.distributed.algorithms.ddp_comm_hooks import powerSGD_hook as powerSGD

model = MyModel()
trainer = Trainer(
    gpus=4,
    plugins=DDPPlugin(
        ddp_comm_state=powerSGD.PowerSGDState(
            process_group=None,
            matrix_approximation_rank=1,
            start_powerSGD_iter=5000,
        ),
        ddp_comm_hook=powerSGD.powerSGD_hook,
    ),
)
trainer.fit(model)

组合挂钩以获得累积收益：

> DDP 通信包装器需要 pytorch 版本至少为 1.9.0

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.plugins import DDPPlugin
from torch.distributed.algorithms.ddp_comm_hooks import (
    default_hooks as default,
    powerSGD_hook as powerSGD,
)

model = MyModel()
trainer = Trainer(
    gpus=4,
    plugins=DDPPlugin(
        ddp_comm_state=powerSGD.PowerSGDState(
            process_group=None,
            matrix_approximation_rank=1,
            start_powerSGD_iter=5000,
        ),
        ddp_comm_hook=powerSGD.powerSGD_hook,
        ddp_comm_wrapper=default.fp16_compress_wrapper,
    ),
)
trainer.fit(model)

# 混合精度训练

混合精度结合使用 FP32 和低位浮点（如 FP16）来减少模型训练期间的内存占用，从而提高性能。

Lightning 提供针对 GPU 和 CPU 的混合精度训练，以及针对 TPU 的 bfloat16 混合精度训练。

> 在某些情况下，为了数值稳定性保持在 FP32 中很重要，因此在使用混合精度时请记住这一点。
>
> 例如，在前向（例如 torchpoint3d）计算期间运行分散操作时，必须保留在 FP32 中。

## FP16 混合精度

在大多数情况下，混合精度使用 FP16。 支持的 Torch 操作在 FP16 中自动运行，从而节省内存并提高 GPU 和 TPU 加速器的吞吐量。

由于计算是在 FP16 中进行的，因此存在数值不稳定的可能性。 这是由动态梯度缩放器内部处理的，它跳过无效的步骤，并调整缩放器以确保后续步骤落在有限范围内。 有关更多信息，请参阅自动广播文档。

> 使用 TPU 时，设置 precision=16 将启用 bfloat16，这是 TPU 上唯一支持的精度类型。

In [ ]:
Trainer(gpus=1, precision=16)

## BFloat16 混合精度

> BFloat16 需要 PyTorch 1.10 或更高版本。 目前这需要每晚安装 PyTorch。  
> BFloat16 也是实验性的，可能不会提供大的加速或内存改进，但提供更好的数值稳定性。  
> 请注意 GPU，最大的好处需要基于安培的 GPU，例如 A100s 或 3090s。  

BFloat16 混合精度类似于 FP16 混合精度，但是我们保留了更多 FP32 必须提供的“动态范围”。 这意味着与 FP16 混合精度相比，我们能够提高数值稳定性。 有关更多信息，请参阅此 TPU 性能博客文章。

由于 BFloat16 在训练过程中比 FP16 更稳定，因此我们无需担心使用 FP16 混合精度带来的任何梯度缩放或 nan 梯度值。

In [ ]:
Trainer(gpus=1, precision="bf16")

也可以在 CPU 上使用 BFloat16 混合精度，依赖于引擎盖下的 MKLDNN。

In [ ]:
Trainer(precision="bf16")

## NVIDIA APEX 混合精度

> 我们强烈建议使用上述原生混合精度而不是 NVIDIA APEX，除非您需要更精细的控制。

NVIDIA APEX 在设置混合精度方面提供了一些额外的灵活性。 这在想要尝试不同的精度配置时很有用，例如将大部分权重保留在 FP16 中以及在 FP16 中运行计算。

In [ ]:
Trainer(gpus=1, amp_backend="apex")

通过训练器设置 NVIDIA 优化级别。

In [ ]:
Trainer(gpus=1, amp_backend="apex", amp_level="O2")

# 保存和装载重量

Lightning 自动保存和加载检查点。 检查点捕获模型使用的所有参数的确切值。

检查点训练允许您在训练过程中断时恢复训练过程、微调模型或使用预先训练的模型进行推理，而无需重新训练模型。

## 检查点保存

闪电检查点拥有恢复培训课程所需的一切，包括：
* 16 位缩放因子（顶点）
* 当前时代
* 全局步骤
* 模型 state_dict
* 所有优化器的状态
* 所有 learningRate 调度程序的状态
* 所有回调的状态
* 如果作为 hparams (Argparse.Namespace) 传入，则用于该模型的超参数

## 自动保存

Lightning 会自动在您当前的工作目录中为您保存一个检查点，其中包含您上次训练时期的状态。 这可确保您可以在培训中断时恢复培训。

要更改检查点路径传入：

In [ ]:
# saves checkpoints to '/your/path/to/save/checkpoints' at every epoch end
trainer = Trainer(default_root_dir="/your/path/to/save/checkpoints")

您可以自定义检查点行为以监控任何数量的训练或验证步骤。 例如，如果您想根据验证损失更新检查点：
* 计算您希望监控的任何指标或其他数量，例如验证损失。
* 使用 log() 方法记录数量，并使用诸如 val_loss 之类的键。
* 初始化ModelCheckpoint回调，设置monitor为你的数量key。
* 将回调传递给回调训练器标志。

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint


class LitAutoEncoder(LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)

        # 1. calculate loss
        loss = F.cross_entropy(y_hat, y)

        # 2. log `val_loss`
        self.log("val_loss", loss)


# 3. 初始化 ModelCheckpoint 回调，监控 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor="val_loss")

# 4. 将您的回调添加到回调列表中
trainer = Trainer(callbacks=[checkpoint_callback])

您还可以控制更高级的选项，如 save_top_k，以保存最佳 k 模型和监控数量（最小/最大）的模式，save_weights_only 或 period 以设置检查点之间的 epoch 间隔，以避免减速。

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint


class LitAutoEncoder(LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)


# saves a file like: my/path/sample-mnist-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="my/path/",
    filename="sample-mnist-{epoch:02d}-{val_loss:.2f}",
    save_top_k=3,
    mode="min",
)

trainer = Trainer(callbacks=[checkpoint_callback])

您可以通过调用在训练后检索检查点

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath="my/path/")
trainer = Trainer(callbacks=[checkpoint_callback])
trainer.fit(model)
checkpoint_callback.best_model_path

### 禁用检查点

您可以通过传递禁用检查点

In [ ]:
trainer = Trainer(checkpoint_callback=False)

Lightning 检查点还将传入 LightningModule init 的参数保存在检查点中的 hyper_parameters 键下。

In [ ]:
class MyLightningModule(LightningModule):
    def __init__(self, learning_rate, *args, **kwargs):
        super().__init__()
        self.save_hyperparameters()


# 所有 init args 都保存到检查点
checkpoint = torch.load(CKPT_PATH)
print(checkpoint["hyper_parameters"])
# {'learning_rate': the_value}

## 手动保存

您可以手动保存检查点并从检查点状态恢复模型。

In [ ]:
model = MyLightningModule(hparams)
trainer.fit(model)
trainer.save_checkpoint("example.ckpt")
new_model = MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")

## 使用加速器手动保存

Lightning 还处理运行多个进程的加速器，例如 DDP。 例如，当使用 DDP 加速器时，我们的训练脚本同时在多个设备上运行。 Lightning 自动确保模型仅保存在主进程中，而其他进程不会干扰保存检查点。 这不需要更改代码，如下所示。

In [ ]:
trainer = Trainer(accelerator="ddp")
model = MyLightningModule(hparams)
trainer.fit(model)
# 仅保存在主进程上
trainer.save_checkpoint("example.ckpt")

不使用 trainer.save_checkpoint 会导致意外行为和潜在的死锁。 使用其他保存功能将导致所有设备尝试保存检查点。 因此，我们强烈建议您使用训练器的保存功能。 如果无法避免使用自定义保存函数，我们建议使用 rank_zero_only() 以确保保存仅发生在主进程上。

## 检查点加载

要加载模型及其权重、偏差和超参数，请使用以下方法：

In [ ]:
model = MyLightingModule.load_from_checkpoint(PATH)

print(model.learning_rate)
# prints the learning_rate you used in this checkpoint

model.eval()
y_hat = model(x)

但是如果你不想使用检查点中保存的值，在这里传入你自己的值

In [ ]:
class LitModel(LightningModule):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.save_hyperparameters()
        self.l1 = nn.Linear(self.hparams.in_dim, self.hparams.out_dim)

你可以像这样恢复模型

In [ ]:
# 如果你像这样训练和保存模型，它会在加载权重时使用这些值。 但是你可以覆盖这个
LitModel(in_dim=32, out_dim=10)

# uses in_dim=32, out_dim=10
model = LitModel.load_from_checkpoint(PATH)

# uses in_dim=128, out_dim=10
model = LitModel.load_from_checkpoint(PATH, in_dim=128, out_dim=10)

In [ ]:
# load weights without mapping ...
MyLightningModule.load_from_checkpoint('path/to/checkpoint.ckpt')

# or load weights mapping all weights from GPU 1 to GPU 0 ...
map_location = {'cuda:1':'cuda:0'}
MyLightningModule.load_from_checkpoint(
    'path/to/checkpoint.ckpt',
    map_location=map_location
)

# or load weights and hyperparameters from separate files.
MyLightningModule.load_from_checkpoint(
    'path/to/checkpoint.ckpt',
    hparams_file='/path/to/hparams_file.yaml'
)

# override some of the params with new values
MyLightningModule.load_from_checkpoint(
    PATH,
    num_layers=128,
    pretrained_ckpt_path: NEW_PATH,
)

# predict
pretrained_model.eval()
pretrained_model.freeze()
y_hat = pretrained_model(x)

## 恢复训练状态

如果您不只是想加载权重，而是要恢复完整的训练，请执行以下操作：

In [ ]:
model = LitModel()
trainer = Trainer(resume_from_checkpoint="some/path/to/my_checkpoint.ckpt")

# 自动恢复模型、纪元、步骤、LR 调度程序、顶点等...
trainer.fit(model)

# 优化

Lightning 提供两种模式来管理优化过程：
* 自动优化
* 手动优化

对于大多数研究案例，自动优化将为您做正确的事情，这是大多数用户应该使用的。

对于想要进行深奥的优化计划或技术的高级/专家用户，请使用手动优化。

## 手动优化

对于强化学习、稀疏编码或 GAN 研究等高级研究主题，可能需要手动管理优化过程。

这仅推荐给需要最大灵活性的专家。 Lightning 将仅处理精度和加速器逻辑。 用户剩下的是 optimizer.zero_grad()、梯度累积、模型切换等。

要手动优化，请执行以下操作：
* 在 LightningModule 的 __init__ 中设置 self.automatic_optimization=False。
* 使用以下函数并手动调用它们：
    * self.optimizers() 访问您的优化器（一个或多个）
    * optimizer.zero_grad() 清除前一个训练步骤中的梯度
    * self.manual_backward(loss) 而不是 loss.backward()
    * optimizer.step() 更新您的模型参数

这是手动优化的最小示例。

In [ ]:
from pytorch_lightning import LightningModule


class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        # 重要提示：此属性激活手动优化。
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        opt = self.optimizers()
        opt.zero_grad()
        loss = self.compute_loss(batch)
        self.manual_backward(loss)
        opt.step()

> 在调用 optimizer.zero_grad() 的地方要小心，否则你的模型不会收敛。 在 self.manual_backward(loss) 之前调用 optimizer.zero_grad() 是一种很好的做法。

## 梯度积累

您可以在批次上累积梯度，类似于自动优化的cumulative_grad_batches。 要使用一个优化器执行梯度累积，您可以这样做。

In [ ]:
# 在 n 个批次上累积梯度
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def training_step(self, batch, batch_idx):
    opt = self.optimizers()

    loss = self.compute_loss(batch)
    self.manual_backward(loss)

    # 累积 `n` 个批次的梯度
    if (batch_idx + 1) % n == 0:
        opt.step()
        opt.zero_grad()

## 使用多个优化器（如 GAN）[手册]

这是一个使用多个优化器训练简单 GAN 的示例。

In [ ]:
import torch
from torch import Tensor
from pytorch_lightning import LightningModule


class SimpleGAN(LightningModule):
    def __init__(self):
        super().__init__()
        self.G = Generator()
        self.D = Discriminator()

        # 重要提示：此属性激活手动优化。
        self.automatic_optimization = False

    def sample_z(self, n) -> Tensor:
        sample = self._Z.sample((n,))
        return sample

    def sample_G(self, n) -> Tensor:
        z = self.sample_z(n)
        return self.G(z)

    def training_step(self, batch, batch_idx):
        # Implementation follows the PyTorch tutorial:
        # https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        g_opt, d_opt = self.optimizers()

        X, _ = batch
        batch_size = X.shape[0]

        real_label = torch.ones((batch_size, 1), device=self.device)
        fake_label = torch.zeros((batch_size, 1), device=self.device)

        g_X = self.sample_G(batch_size)

        ##########################
        # Optimize Discriminator #
        ##########################
        d_x = self.D(X)
        errD_real = self.criterion(d_x, real_label)

        d_z = self.D(g_X.detach())
        errD_fake = self.criterion(d_z, fake_label)

        errD = errD_real + errD_fake

        d_opt.zero_grad()
        self.manual_backward(errD)
        d_opt.step()

        ######################
        # Optimize Generator #
        ######################
        d_z = self.D(g_X)
        errG = self.criterion(d_z, real_label)

        g_opt.zero_grad()
        self.manual_backward(errG)
        g_opt.step()

        self.log_dict({"g_loss": errG, "d_loss": errD}, prog_bar=True)

    def configure_optimizers(self):
        g_opt = torch.optim.Adam(self.G.parameters(), lr=1e-5)
        d_opt = torch.optim.Adam(self.D.parameters(), lr=1e-5)
        return g_opt, d_opt

## 学习率调度

您使用的每个优化器都可以与任何学习率调度器配对。 有关所有可用选项，请参阅 configure_optimizers() 的文档

## 学习率调度 [手册]

您可以以任意时间间隔调用 lr_scheduler.step()。 在 LightningModule 中使用 self.lr_schedulers() 访问在 configure_optimizers() 中定义的任何学习率调度程序。

> 在 1.3 之前，Lightning 在自动和手动优化中都会自动调用 lr_scheduler.step()。 从 1.3 开始， lr_scheduler.step() 现在可供用户以任意时间间隔调用。  
> 请注意，即使在手动优化期间在 configure_optimizers() 中提供了 lr_dict 键，例如“step”和“interval”，它们也会被忽略。

这是一个每一步都调用 lr_scheduler.step() 的示例。

In [ ]:
# step every batch
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def training_step(self, batch, batch_idx):
    # do forward, backward, and optimization
    ...

    # single scheduler
    sch = self.lr_schedulers()
    sch.step()

    # multiple schedulers
    sch1, sch2 = self.lr_schedulers()
    sch1.step()
    sch2.step()

如果您想每 n 个步骤/时期调用 lr_scheduler.step()，请执行以下操作。

In [ ]:
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def training_step(self, batch, batch_idx):
    # do forward, backward, and optimization
    ...

    sch = self.lr_schedulers()

    # step every `n` batches
    if (batch_idx + 1) % n == 0:
        sch.step()

    # step every `n` epochs
    if self.trainer.is_last_batch and (self.trainer.current_epoch + 1) % n == 0:
        sch.step()

如果要在每个 epoch 后调用需要度量值的调度程序，请考虑执行以下操作：

In [ ]:
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def training_epoch_end(self, outputs):
    sch = self.lr_schedulers()

    # If the selected scheduler is a ReduceLROnPlateau scheduler.
    if isinstance(sch, torch.optim.lr_scheduler.ReduceLROnPlateau):
        sch.step(self.trainer.callback_metrics["loss"])

## 对 LBFGS 类优化器使用闭包

为优化器提供一个闭包函数来执行模型的前向、零梯度和后向是一个很好的做法。 对于大多数优化器来说，它是可选的，但如果您切换到需要闭包的优化器（例如 torch.optim.LBFGS），则会使您的代码兼容。

有关闭包的更多信息，请参阅 PyTorch 文档。

这是一个使用闭包函数的例子。

In [ ]:
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def configure_optimizers(self):
    return torch.optim.LBFGS(...)


def training_step(self, batch, batch_idx):
    opt = self.optimizers()

    def closure():
        loss = self.compute_loss(batch)
        opt.zero_grad()
        self.manual_backward(loss)
        return loss

    opt.step(closure=closure)

## 访问您自己的优化器 [手册]

优化器是一个 LightningOptimizer 对象，它包含在你的 configure_optimizers() 中配置的你自己的优化器。 您可以使用 optimizer.optimizer 访问您自己的优化器。 但是，如果您使用自己的优化器来执行一个步骤，Lightning 将无法为您提供加速器和精度支持。

In [ ]:
def __init__(self):
    super().__init__()
    self.automatic_optimization = False


def training_step(batch, batch_idx):
    optimizer = self.optimizers()

    # `optimizer` is a `LightningOptimizer` wrapping the optimizer.
    # To access it, do the following.
    # However, it won't work on TPU, AMP, etc...
    optimizer = optimizer.optimizer
    ...

## 自动优化

使用 Lightning，大多数用户不必考虑何时调用 .zero_grad()、.backward() 和 .step()，因为 Lightning 会为您自动化。

在幕后，Lightning 执行以下操作：

In [ ]:
for epoch in epochs:
    for batch in data:

        def closure():
            loss = model.training_step(batch, batch_idx, ...)
            optimizer.zero_grad()
            loss.backward()
            return loss

        optimizer.step(closure)

    for lr_scheduler in lr_schedulers:
        lr_scheduler.step()

在多个优化器的情况下，Lightning 会执行以下操作：

In [ ]:
for epoch in epochs:
    for batch in data:
        for opt in optimizers:

            def closure():
                loss = model.training_step(batch, batch_idx, optimizer_idx)
                opt.zero_grad()
                loss.backward()
                return loss

            opt.step(closure)

    for lr_scheduler in lr_schedulers:
        lr_scheduler.step()

从上面的代码片段中可以看出，Lightning 定义了一个带有 training_step、zero_grad 和向后的闭包，供优化器执行。 这种机制是为了支持优化器对闭包的输出（例如损失）进行操作或需要多次调用闭包（例如 LBFGS）。

> 在 1.2.2 之前，Lightning 内部按顺序调用了backward、step 和 zero_grad。 从 1.2.2 开始，顺序改为 zero_grad、backward 和 step。

### 使用多个优化器（如 GAN）

要使用多个优化器（可选择使用学习率调度器），请从 configure_optimizers() 返回两个或多个优化器。

In [ ]:
# 两个优化器，没有调度器
def configure_optimizers(self):
    return Adam(...), SGD(...)


# 两个优化器，一个仅用于 adam 的调度器
def configure_optimizers(self):
    opt1 = Adam(...)
    opt2 = SGD(...)
    optimizers = [opt1, opt2]
    lr_schedulers = {"scheduler": ReduceLROnPlateau(opt1, ...), "monitor": "metric_to_track"}
    return optimizers, lr_schedulers


# 两个优化器，两个调度器
def configure_optimizers(self):
    opt1 = Adam(...)
    opt2 = SGD(...)
    return [opt1, opt2], [StepLR(opt1, ...), OneCycleLR(opt2, ...)]

在幕后，Lightning 将依次调用每个优化器：

In [ ]:
for epoch in epochs:
    for batch in data:
        for opt in optimizers:
            loss = train_step(batch, batch_idx, optimizer_idx)
            opt.zero_grad()
            loss.backward()
            opt.step()

    for lr_scheduler in lr_schedulers:
        lr_scheduler.step()

## 以任意间隔步进优化器

要使用优化器做更多有趣的事情，例如学习率预热或奇数调度，请覆盖 optimizer_step() 函数。

> 如果您要覆盖此方法，请确保将 optimizer_closure 参数传递给 optimizer.step() 函数，如示例中所示，因为在闭包函数中调用了 training_step()、optimizer.zero_grad()、backward()。

例如，这里每批次步进优化器 A，每 2 批次步进优化器 B。

In [ ]:
# 优化器步骤的交替计划（例如 GAN）
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,b
    using_native_amp=False,
    using_lbfgs=False,
):
    # update generator every step
    if optimizer_idx == 0:
        optimizer.step(closure=optimizer_closure)

    # update discriminator every 2 steps
    if optimizer_idx == 1:
        if (batch_idx + 1) % 2 == 0:
            # the closure (which includes the `training_step`) will be executed by `optimizer.step`
            optimizer.step(closure=optimizer_closure)
        else:
            # optional: call the closure by itself to run `training_step` + `backward` without an optimizer step
            optimizer_closure()

    # ...
    # add as many optimizers as you want

在这里，我们添加了学习率预热。

In [ ]:
# learning rate warm-up
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,
    using_native_amp=False,
    using_lbfgs=False,
):
    # skip the first 500 steps
    if self.trainer.global_step < 500:
        lr_scale = min(1.0, float(self.trainer.global_step + 1) / 500.0)
        for pg in optimizer.param_groups:
            pg["lr"] = lr_scale * self.hparams.learning_rate

    # update params
    optimizer.step(closure=optimizer_closure)

## 访问您自己的优化器

优化器是一个 LightningOptimizer 对象，它包含在你的 configure_optimizers() 中配置的你自己的优化器。 您可以使用 optimizer.optimizer 访问您自己的优化器。 但是，如果您使用自己的优化器来执行一个步骤，Lightning 将无法为您提供加速器和精度支持。

In [ ]:
# function hook in LightningModule
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,
    using_native_amp=False,
    using_lbfgs=False,
):
    optimizer.step(closure=optimizer_closure)


# `optimizer` is a `LightningOptimizer` wrapping the optimizer.
# To access it, do the following.
# However, it won't work on TPU, AMP, etc...
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,
    using_native_amp=False,
    using_lbfgs=False,
):
    optimizer = optimizer.optimizer
    optimizer.step(closure=optimizer_closure)

In [ ]:
# function hook in LightningModule
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,
    using_native_amp=False,
    using_lbfgs=False,
):
    optimizer.step(closure=optimizer_closure)


# `optimizer` is a `LightningOptimizer` wrapping the optimizer.
# To access it, do the following.
# However, it won't work on TPU, AMP, etc...
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu=False,
    using_native_amp=False,
    using_lbfgs=False,
):
    optimizer = optimizer.optimizer
    optimizer.step(closure=optimizer_closure)

# 性能和瓶颈分析器

分析您的训练运行可以帮助您了解代码中是否存在任何瓶颈。

## 内置检查

PyTorch Lightning 支持开箱即用地分析训练循环中的标准动作，包括：
* on_epoch_start
* on_epoch_end
* on_batch_start
* tbptt_split_batch
* model_forward
* model_backward
* on_after_backward
* optimizer_step
* on_batch_end
* training_step_end
* on_training_end

## 启用简单分析

如果您只想分析标准动作，您可以在构建 Trainer 对象时设置 profiler=”simple”。

In [ ]:
trainer = Trainer(..., profiler="simple")

分析器的结果将在训练 fit() 完成时打印。

In [ ]:
Profiler Report

Action                  |  Mean duration (s)    |  Total time (s)
-----------------------------------------------------------------
on_epoch_start          |  5.993e-06            |  5.993e-06
get_train_batch         |  0.0087412            |  16.398
on_batch_start          |  5.0865e-06           |  0.0095372
model_forward           |  0.0017818            |  3.3408
model_backward          |  0.0018283            |  3.4282
on_after_backward       |  4.2862e-06           |  0.0080366
optimizer_step          |  0.0011072            |  2.0759
on_batch_end            |  4.5202e-06           |  0.0084753
on_epoch_end            |  3.919e-06            |  3.919e-06
on_train_end            |  5.449e-06            |  5.449e-06

## 高级分析

如果您想了解每个事件期间调用的函数的更多信息，可以使用 AdvancedProfiler。 此选项使用 Python 的 cProfiler 来提供在代码中调用的每个函数上花费的时间的报告。

In [ ]:
trainer = Trainer(..., profiler="advanced")

# or

profiler = AdvancedProfiler()
trainer = Trainer(..., profiler=profiler)

分析器的结果将在训练 fit() 完成时打印。 此分析器报告可能很长，因此您还可以指定一个 output_filename 来保存报告，而不是将其记录到终端的输出中。 下面的输出显示了操作 get_train_batch 的分析。

In [ ]:
Profiler Report

Profile stats for: get_train_batch
        4869394 function calls (4863767 primitive calls) in 18.893 seconds
Ordered by: cumulative time
List reduced from 76 to 10 due to restriction <10>
ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3752/1876    0.011    0.000   18.887    0.010 {built-in method builtins.next}
    1876     0.008    0.000   18.877    0.010 dataloader.py:344(__next__)
    1876     0.074    0.000   18.869    0.010 dataloader.py:383(_next_data)
    1875     0.012    0.000   18.721    0.010 fetch.py:42(fetch)
    1875     0.084    0.000   18.290    0.010 fetch.py:44(<listcomp>)
    60000    1.759    0.000   18.206    0.000 mnist.py:80(__getitem__)
    60000    0.267    0.000   13.022    0.000 transforms.py:68(__call__)
    60000    0.182    0.000    7.020    0.000 transforms.py:93(__call__)
    60000    1.651    0.000    6.839    0.000 functional.py:42(to_tensor)
    60000    0.260    0.000    5.734    0.000 transforms.py:167(__call__)

您还可以在 LightningModule 中引用此分析器来分析感兴趣的特定操作。 如果您不想始终打开分析器，您可以选择传递 PassThroughProfiler，它允许您跳过分析而无需进行任何代码更改。 每个分析器都有一个方法 profile() ，它返回一个上下文处理程序。 只需传入您要跟踪的操作的名称，分析器将记录在此上下文中执行的代码的性能。

In [ ]:
from pytorch_lightning.profiler import Profiler, PassThroughProfiler


class MyModel(LightningModule):
    def __init__(self, profiler=None):
        self.profiler = profiler or PassThroughProfiler()

    def custom_processing_step(self, data):
        with profiler.profile("my_custom_action"):
            ...
        return data


profiler = Profiler()
model = MyModel(profiler)
trainer = Trainer(profiler=profiler, max_epochs=1)

## PyTorch 分析

Autograd 包含一个分析器，可让您检查模型中不同运算符的成本 - 在 CPU 和 GPU 上。

要阅读有关 PyTorch Profiler 及其所有选项的更多信息，请查看其文档

In [ ]:
trainer = Trainer(..., profiler="pytorch")

# or

profiler = PyTorchProfiler(...)
trainer = Trainer(..., profiler=profiler)

此分析器与 PyTorch DistributedDataParallel 配合使用。 如果提供了文件名，每个等级都会将他们的分析操作保存到他们自己的文件中。 分析器报告可能很长，因此您设置文件名将保存报告，而不是将其记录到终端的输出中。 如果没有给出文件名，它只会记录在 0 级上。

分析器的结果将在 {fit,validate,test,predict} 完成时打印。

默认情况下，此分析器将记录 training_step_and_backward、training_step、backward、validation_step、test_step 和 predict_step。 下面的输出显示了操作 training_step_and_backward 的分析。 用户可以提供 PyTorchProfiler(record_functions={...}) 来扩展分析函数的范围。

> 使用 PyTorch Profiler 时，挂钟时间不能代表真正的挂钟时间。 这是因为当许多 CUDA 操作异步发生时，强制同步测量分析操作。 建议使用此 Profiler 来查找瓶颈/故障，但是对于端到端挂钟时间，请使用 SimpleProfiler。 # noqa：E501

In [ ]:
Profiler Report

Profile stats for: training_step_and_backward
---------------------  ---------------  ---------------  ---------------  ---------------  ---------------
Name                   Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg
---------------------  ---------------  ---------------  ---------------  ---------------  ---------------
t                      62.10%           1.044ms          62.77%           1.055ms          1.055ms
addmm                  32.32%           543.135us        32.69%           549.362us        549.362us
mse_loss               1.35%            22.657us         3.58%            60.105us         60.105us
mean                   0.22%            3.694us          2.05%            34.523us         34.523us
div_                   0.64%            10.756us         1.90%            32.001us         16.000us
ones_like              0.21%            3.461us          0.81%            13.669us         13.669us
sum_out                0.45%            7.638us          0.74%            12.432us         12.432us
transpose              0.23%            3.786us          0.68%            11.393us         11.393us
as_strided             0.60%            10.060us         0.60%            10.060us         3.353us
to                     0.18%            3.059us          0.44%            7.464us          7.464us
empty_like             0.14%            2.387us          0.41%            6.859us          6.859us
empty_strided          0.38%            6.351us          0.38%            6.351us          3.175us
fill_                  0.28%            4.782us          0.33%            5.566us          2.783us
expand                 0.20%            3.336us          0.28%            4.743us          4.743us
empty                  0.27%            4.456us          0.27%            4.456us          2.228us
copy_                  0.15%            2.526us          0.15%            2.526us          2.526us
broadcast_tensors      0.15%            2.492us          0.15%            2.492us          2.492us
size                   0.06%            0.967us          0.06%            0.967us          0.484us
is_complex             0.06%            0.961us          0.06%            0.961us          0.481us
stride                 0.03%            0.517us          0.03%            0.517us          0.517us
---------------------  ---------------  ---------------  ---------------  ---------------  ---------------
Self CPU time total: 1.681ms

使用 PyTorchProfiler(emit_nvtx=True) 运行时。 您应该按以下方式运行：

In [ ]:
nvprof --profile-from-start off -o trace_name.prof -- <regular command here>

要可视化分析的操作，您可以：

用：

In [ ]:
nvvp trace_name.prof

或者：

In [ ]:
python -c 'import torch; print(torch.autograd.profiler.load_nvprof("trace_name.prof"))'

# 序列数据

## 通过时间截断反向传播

有时每个批次都需要多次向后传递。 例如，在训练 RNN 时使用 Truncated Backpropagation Through Time 可以节省内存。

Lightning 可以通过这个标志自动处理 TBTT。

In [ ]:
from pytorch_lightning import LightningModule


class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        # 重要：这个属性激活了时间截断的反向传播
        # 将此值设置为 2 会将批次拆分为大小为 2 的序列
        self.truncated_bptt_steps = 2

    # 通过时间截断反向传播
    def training_step(self, batch, batch_idx, hiddens):
        # 必须更新训练步骤以接受 ``hiddens`` 参数 hiddens 是前一个被截断的反向传播步骤的隐藏
        out, hiddens = self.lstm(data, hiddens)
        return {"loss": ..., "hiddens": hiddens}

如果您需要修改批处理的拆分方式，请覆盖 pytorch_lightning.core.LightningModule.tbptt_split_batch()。

# 单 GPU 训练

确保你在一台至少有一个 GPU 的机器上运行。 Lightning 会为您处理所有 NVIDIA 标志，您无需自行设置。

In [ ]:
# train on 1 GPU (using dp mode)
trainer = Trainer(gpus=1)

# 训练技巧

闪电在训练期间实施各种技巧以提供帮助

## 累积梯度

累积梯度在进行反向传递之前运行 K 个大小为 N 的小批次。 效果是大小为 KxN 的大有效批量大小。

In [ ]:
# DEFAULT (ie: no accumulated grads)
trainer = Trainer(accumulate_grad_batches=1)

## 梯度剪裁

可以启用梯度裁剪以避免梯度爆炸。 默认情况下，这将通过调用在所有模型参数上计算的 torch.nn.utils.clip_grad_norm_() 来裁剪梯度范数。 如果 Trainer 的 gradient_clip_algorithm 设置为“value”（默认为“norm”），这将改为使用 torch.nn.utils.clip_grad_norm_() 代替每个参数。

> 如果使用混合精度，则不需要更改gradient_clip_val，因为在应用裁剪函数之前梯度未缩放。

In [ ]:
# DEFAULT (ie: don't clip)
trainer = Trainer(gradient_clip_val=0)

# clip gradients' global norm to <=0.5
trainer = Trainer(gradient_clip_val=0.5)  # gradient_clip_algorithm='norm' by default

# clip gradients' maximum magnitude to <=0.5
trainer = Trainer(gradient_clip_val=0.5, gradient_clip_algorithm="value")

## 随机加权平均

随机权重平均 (SWA) 可以使您的模型更好地泛化，而且几乎无需额外成本。 这可以用于非训练模型和训练模型。 SWA 程序平滑了损失情况，因此在优化过程中更难最终达到局部最小值。

有关 SWA 及其工作原理的更详细说明，请阅读 PyTorch 团队的这篇文章。

In [ ]:
# 启用随机权重平均 - 使用类默认值
trainer = Trainer(stochastic_weight_avg=True)

# 或者，如果您需要传递自定义参数
trainer = Trainer(callbacks=[StochasticWeightAveraging(...)])

## 批量大小的自动缩放

可以启用批量大小的自动缩放以找到适合内存的最大批量大小。 更大的批量通常会产生更好的梯度估计，但也可能导致更长的训练时间。 灵感来自 https://github.com/BlackHC/toma。

In [ ]:
# DEFAULT (ie: don't scale batch size automatically)
trainer = Trainer(auto_scale_batch_size=None)

# Autoscale batch size
trainer = Trainer(auto_scale_batch_size=None | "power" | "binsearch")

# find the batch size
trainer.tune(model)

目前，此功能支持两种模式“power”缩放和“binsearch”缩放。 在“功率”缩放中，从批量大小 1 开始，批量大小不断加倍，直到遇到内存不足 (OOM) 错误。 将参数设置为 ‘binsearch’ 最初也会尝试将批量大小加倍，直到遇到 OOM，之后它将执行二进制搜索以微调批量大小。 此外，应注意批量大小缩放器无法搜索大于训练数据集大小的批量大小。

> 此功能期望 batch_size 字段位于模型属性中，即 model.batch_size 或 hparams 中的字段，即 model.hparams.batch_size。 该字段应该存在，并将被该算法的结果覆盖。 此外，您的 train_dataloader() 方法应依赖此字段才能使此功能正常工作，即
> ```python
def train_dataloader(self):
    return DataLoader(train_dataset, batch_size=self.batch_size | self.hparams.batch_size)
```

> 由于这些限制，当将数据加载器直接传递给 .fit() 时，此功能不起作用。

缩放算法有许多参数，用户可以通过调用 scale_batch_size() 方法来控制这些参数：

In [ ]:
# Use default in trainer construction
trainer = Trainer()
tuner = Tuner(trainer)

# Invoke method
new_batch_size = tuner.scale_batch_size(model, *extra_parameters_here)

# Override old batch size (this is done automatically)
model.hparams.batch_size = new_batch_size

# Fit as normal
trainer.fit(model)

简短的算法通过以下方式工作：
1. 转储模型和训练器的当前状态
2. 迭代直到达到收敛或最大尝试次数 max_trials（默认 25）：
    * 调用训练器的 fit() 方法。 这将评估steps_per_trial（默认为3）训练步骤的数量。 如果在步骤期间分配的张量（训练批次、权重、梯度等）具有太大的内存占用，则每个训练步骤都可能触发 OOM 错误。
    * 如果遇到 OOM 错误，请减小批量大小，否则增加它。 批量大小的增加/减少程度由所选策略决定。
3. 找到的批量大小保存到 model.batch_size 或 model.hparams.batch_size
4. 恢复模型和训练器的初始状态

> DDP 或其任何变体尚不支持批量大小查找器，它即将推出。

## 高级 GPU 优化

在单台或多台 GPU 机器上训练时，Lightning 提供了一系列高级优化来提高吞吐量、内存效率和模型扩展。 有关更多详细信息，请参阅高级 GPU 优化训练。

# 修剪和量化

修剪和量化是压缩模型大小以进行部署的技术，可以在不显着损失精度的情况下加快推理速度和节能。

## 修剪

修剪是一种专注于消除一些模型权重以减小模型大小并降低推理要求的技术。

修剪已被证明可以显着提高效率，同时最大限度地减少模型性能（预测质量）的下降。 建议对云端点、在边缘设备上部署模型或移动推理（等等）进行模型修剪。

要在 Lightning 训练期间启用修剪，只需将 ModelPruning 回调传递给 Lightning Trainer。 PyTorch 的原生修剪实现在幕后使用。

此回调支持多个修剪函数：将任何 `torch.nn.utils.prune` 函数作为字符串传递以选择要修剪的权重（`random_unstructured`、`RandomStructured` 等）或通过子类化 BasePruningMethod 实现您自己的权重。。

In [ ]:
from pytorch_lightning.callbacks import ModelPruning

# 将数量设置为要修剪的参数的分数
trainer = Trainer(callbacks=[ModelPruning("l1_unstructured", amount=0.5)])

您还可以执行迭代修剪、应用彩票假设等等！

In [ ]:
def compute_amount(epoch):
    # 所有返回值的总和需要小于 1
    if epoch == 10:
        return 0.5

    elif epoch == 50:
        return 0.25

    elif 75 < epoch < 99:
        return 0.01


# 金额也可以是可调用的
trainer = Trainer(callbacks=[ModelPruning("l1_unstructured", amount=compute_amount)])

## 量化

量化处于测试阶段，可能会发生变化。

模型量化是另一种性能优化技术，它允许通过以比浮点精度更低的位宽（例如 INT8 或 FLOAT16）执行计算和存储张量来加速推理并降低内存需求。这在模型部署期间特别有益。

量化感知训练 (QAT) 模拟训练期间量化的效果：计算以浮点精度进行，但会考虑后续的量化效果。训练完成后，权重和激活被量化为较低的精度，仅用于推理。

当需要在内存有限的机器上为大型模型提供服务时，或者当需要在模型之间切换并且减少 I/O 时间很重要时，量化非常有用。例如，在跨多种语言的单语语音识别模型之间切换。

Lightning 包括 QuantizationAwareTraining 回调（使用 PyTorch 的原生量化，在此处阅读更多信息），它允许创建完全量化的模型（与 torchscript 兼容）。

In [ ]:
from pytorch_lightning.callbacks import QuantizationAwareTraining


class RegressionModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_0 = nn.Linear(16, 64)
        self.layer_0a = torch.nn.ReLU()
        self.layer_1 = nn.Linear(64, 64)
        self.layer_1a = torch.nn.ReLU()
        self.layer_end = nn.Linear(64, 1)

    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_0a(x)
        x = self.layer_1(x)
        x = self.layer_1a(x)
        x = self.layer_end(x)
        return x


trainer = Trainer(callbacks=[QuantizationAwareTraining()])
qmodel = RegressionModel()
trainer.fit(qmodel, ...)

batch = iter(my_dataloader()).next()
qmodel(qmodel.quant(batch[0]))

tsmodel = qmodel.to_torchscript()
tsmodel(tsmodel.quant(batch[0]))

您可以进一步自定义回调：

In [ ]:
qcb = QuantizationAwareTraining(
    # 量化估计质量规范
    observer_type="histogram",
    # 指定哪些层应合并在一起以提高效率
    modules_to_fuse=[(f"layer_{i}", f"layer_{i}a") for i in range(2)],
    # 使您的模型与所有原始输入/输出兼容，在这种情况下，模型被包裹在带有入口/出口层的外壳中。
    input_compatible=True,
)

batch = iter(my_dataloader()).next()
qmodel(batch[0])

# 迁移学习

## 使用预训练模型

有时我们想使用 LightningModule 作为预训练模型。 这很好，因为 LightningModule 只是一个 torch.nn.Module！

> 请记住，LightningModule 完全是 torch.nn.Module，但具有更多功能。

让我们在单独的模型中使用 AutoEncoder 作为特征提取器。

In [ ]:
class Encoder(torch.nn.Module):
    ...


class AutoEncoder(LightningModule):
    def __init__(self):
        self.encoder = Encoder()
        self.decoder = Decoder()


class CIFAR10Classifier(LightningModule):
    def __init__(self):
        # 初始化预训练的 LightningModule
        self.feature_extractor = AutoEncoder.load_from_checkpoint(PATH)
        self.feature_extractor.freeze()

        # 自编码器输出 100-dim 表示，CIFAR-10 有 10 个类
        self.classifier = nn.Linear(100, 10)

    def forward(self, x):
        representations = self.feature_extractor(x)
        x = self.classifier(representations)
        ...

我们使用预训练的自动编码器（LightningModule）进行迁移学习！

## 示例：Imagenet（计算机视觉）

In [ ]:
import torchvision.models as models


class ImagenetTransferLearning(LightningModule):
    def __init__(self):
        super().__init__()

        # 初始化一个预训练的 resnet
        backbone = models.resnet50(pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # 使用预训练模型对 cifar-10（10 个图像类）进行分类
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        ...

微调

In [ ]:
model = ImagenetTransferLearning()
trainer = Trainer()
trainer.fit(model)

并用它来预测您感兴趣的数据

In [ ]:
model = ImagenetTransferLearning.load_from_checkpoint(PATH)
model.freeze()

x = some_images_from_cifar10()
predictions = model(x)

我们在 imagenet 上使用了预训练模型，在 CIFAR-10 上进行了微调，以在 CIFAR-10 上进行预测。 在非学术界，我们会在你拥有的一个小数据集上进行微调，并在你的数据集上进行预测。

## 示例：BERT (NLP)

只要它是 torch.nn.Module 子类，Lightning 与用于迁移学习的内容完全无关。

这是一个使用 Huggingface Transformer 的模型。

In [ ]:
class BertMNLIFinetuner(LightningModule):
    def __init__(self):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-cased", output_attentions=True)
        self.W = nn.Linear(bert.config.hidden_size, 3)
        self.num_classes = 3

    def forward(self, input_ids, attention_mask, token_type_ids):

        h, _, attn = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        h_cls = h[:, 0]
        logits = self.W(h_cls)
        return logits, attn